In [ ]:
# %%
# ==============================================================================
# --- ОСНОВНЫЕ ИМПОРТЫ ДЛЯ АНАЛИЗА (Без Try-Except для опциональных) ---
# ==============================================================================
!pip install pingouin
!pip install factor_analyzer
!pip install xlsxwriter

import datetime
import logging
import re
import traceback
from collections import defaultdict

# Визуализация
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Статистика и машинное обучение (ПРЯМЫЕ ИМПОРТЫ - ОШИБКА ЕСЛИ НЕ УСТАНОВЛЕНО)
import pingouin as pg
import seaborn as sns
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
from IPython.display import Markdown, display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Настройки вывода
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 110)
pd.set_option("display.max_colwidth", 150)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(funcName)s - %(message)s",
)

print("Основные импорты (прямые) и настройки выполнены.")

# --- 2. Параметры ---
file_name = "AI_Perception.xlsx"
sheet1_name = "Блоки 1-3"
sheet2_name = "Блоки 1,4-7"
id_col = "id"  # Исходный ID
final_participant_id_col = "participant_id"


# --- 3. Определения Функций из V37 ---


# Функция Загрузки
def load_sheet(file_path, sheet_name):
    logging.info(f"Загрузка: '{file_path}', лист: '{sheet_name}'")
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        logging.info(f"Успешно. Размер: {df.shape}")
        if id_col in df.columns:
            # Приводим ID к строке и убираем пробелы на всякий случай
            df[id_col] = df[id_col].astype(str).str.strip()
            logging.info(f"Лист '{sheet_name}': {id_col} приведен к строке.")
        else:
            logging.warning(f"Лист '{sheet_name}': Столбец ID '{id_col}' не найден!")
        return df
    except FileNotFoundError:
        logging.error(f"Файл не найден: {file_path}")
        return None
    except ValueError as ve:  # Если лист не найден
        # Проверяем, связано ли это с именем листа
        if f"Worksheet named '{sheet_name}' not found" in str(ve):
            logging.error(f"Лист '{sheet_name}' не найден в файле '{file_path}'.")
        else:
            logging.error(f"Ошибка значения при чтении листа '{sheet_name}': {ve}")
        return None
    except Exception as e:
        logging.error(
            f"Общая ошибка чтения Excel '{file_path}', лист '{sheet_name}': {e}"
        )
        return None


# Функция очистки строк для карт
def _clean_string_for_mapping(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower().strip()
    text = text.replace("\xa0", " ")
    text = text.replace("–", "-").replace("—", "-")
    text = text.replace("ё", "е")
    text = re.sub(r"\(.*?\)", "", text).strip()  # Убираем текст в скобках
    text = text.rstrip(".,:;?!")  # Убираем пунктуацию в конце
    text = re.sub(
        r"(?<=\d)\.(?=\d)", "", text
    )  # Убираем точки между цифрами (1.0 -> 10) - возможно, нужно? Оставил из V37
    text = re.sub(r"\s+", " ", text).strip()  # Нормализуем пробелы
    return text


# Функция генерации суффикса колонки
def _generate_col_suffix(text_to_process, pattern_key_cfg=None):
    if not text_to_process or not isinstance(text_to_process, str):
        return None
    text_clean = str(text_to_process).strip()
    meaningful_text = text_clean

    # Удаляем префикс (номер вопроса + ключ из карты), если он есть
    if pattern_key_cfg:
        pattern_str = str(pattern_key_cfg).strip()
        try:
            # Пытаемся удалить префикс вида "13. ключ_карты " или "ключ_карты "
            pattern_regex = (
                rf"^\s*\d*\s*[\.:\)]?\s*{re.escape(pattern_str)}\s*[:\-]?\s*"
            )
            match_prefix = re.match(
                pattern_regex, text_clean, re.IGNORECASE | re.DOTALL
            )
            if match_prefix:
                meaningful_text = text_clean[match_prefix.end() :].strip()
            else:  # Если не сработало, пробуем просто найти ключ и взять текст после него
                index = text_clean.find(pattern_str)
                if index != -1:
                    meaningful_text = (
                        text_clean[index + len(pattern_str) :].strip().lstrip(":- ")
                    )
        except re.error:
            logging.warning(
                f"Ошибка Regex при удалении паттерна '{pattern_str[:50]}...' из '{text_clean[:50]}...'"
            )
        except Exception as e_gen:
            logging.warning(
                f"Неожиданная ошибка при удалении паттерна '{pattern_str[:50]}...' из '{text_clean[:50]}...': {e_gen}"
            )

    # Дополнительно удаляем префикс вида "1. ", "a) " и т.д. из оставшегося текста
    meaningful_text = re.sub(
        r"^\s*(\d+|[a-zA-Z])\s*[\.:\)]\s*", "", meaningful_text
    ).strip()

    if not meaningful_text:
        return None  # Если после удаления ничего не осталось

    # Очистка и форматирование суффикса
    cleaned = meaningful_text.lower()
    cleaned = cleaned.replace("–", "-").replace("—", "-")
    cleaned = cleaned.replace("\xa0", " ")
    cleaned = cleaned.replace("ё", "е")
    cleaned = re.sub(r"\(.*?\)", "", cleaned).strip()  # Убираем текст в скобках
    cleaned = cleaned.rstrip(".,:;?!")
    cleaned = (
        cleaned.replace(" / ", "_").replace("/", "_").replace(" ", "_")
    )  # Замена разделителей
    cleaned = cleaned.replace("%", "perc")
    cleaned = re.sub(
        r"[^\w\d_]+", "", cleaned, flags=re.UNICODE
    )  # Удаляем все не-буквенно-цифровые символы кроме _
    cleaned = re.sub(r"_+", "_", cleaned).strip(
        "_"
    )  # Убираем двойные подчеркивания и в начале/конце
    cleaned = cleaned[:65]  # Ограничиваем длину

    if cleaned:
        return cleaned
    else:  # Fallback, если все удалилось
        safe_orig = re.sub(r"\W+", "_", text_clean, flags=re.UNICODE).strip("_")[:45]
        suffix = f"fallback_{safe_orig}" if safe_orig else "fallback_suffix"
        logging.warning(
            f"    _generate_col_suffix: Fallback суффикс для '{str(text_to_process)[:60]}' -> '{suffix}'"
        )
        return suffix


# Функция первичной очистки
def initial_cleanup(df, cols_to_drop_patterns, text_block_identifiers):
    logging.info("Первичная очистка...")
    cols_to_drop = []
    # Удаление по паттернам
    for pattern in cols_to_drop_patterns:
        pattern_str = str(pattern).strip()
        if not pattern_str:
            continue
        # Ищем точное совпадение или начало строки
        matched = [
            col
            for col in df.columns
            if str(col).strip() == pattern_str
            or str(col).strip().startswith(pattern_str)
        ]
        cols_to_drop.extend(matched)
    # Удаление текстовых блоков
    for pattern in text_block_identifiers:
        pattern_str = str(pattern).strip()
        if not pattern_str:
            continue
        matched = [
            col for col in df.columns if str(col).strip().startswith(pattern_str)
        ]
        cols_to_drop.extend(matched)
    # Удаление колонок типа "1.", "a."
    misc_to_drop = [
        col
        for col in df.columns
        if re.fullmatch(r"^\s*(\d+|[a-zA-Z])\.\s*$", str(col).strip())
    ]
    cols_to_drop.extend(misc_to_drop)
    # Удаление полностью пустых колонок
    empty_cols = df.columns[df.isna().all()].tolist()
    cols_to_drop.extend(empty_cols)

    # Финальный список уникальных колонок для удаления
    unique_cols_to_drop = sorted(list(set(cols_to_drop)))
    # Убираем те, которых уже нет в DF (на всякий случай)
    valid_cols_to_drop = [c for c in unique_cols_to_drop if c in df.columns]

    if valid_cols_to_drop:
        logging.info(
            f"Удаляем {len(valid_cols_to_drop)} столбцов на этапе очистки: {valid_cols_to_drop[:10]}{'...' if len(valid_cols_to_drop) > 10 else ''}"
        )
        df = df.drop(columns=valid_cols_to_drop, errors="ignore")
        logging.info(f"Размер после первичной очистки: {df.shape}")
    else:
        logging.info("Не найдено столбцов для первичной очистки.")
    return df


# Функция создания очищенных карт
def create_cleaned_map(map_dict):
    if not isinstance(map_dict, dict):
        return map_dict
    return {
        _clean_string_for_mapping(k): v for k, v in map_dict.items() if not pd.isna(k)
    }


# --- Улучшенная функция очистки имен (v2) ---
def clean_col_name_v2(col_name, max_len=60, is_key=False):
    # ... (код clean_col_name_v2 без изменений, одна версия) ...
    if not isinstance(col_name, str):
        col_name = str(col_name)

    # Сначала удаляем префикс типа "Q13_GM_" или "Q8_ActivityField_"
    # Чтобы он не мешал очистке основного текста вопроса
    cleaned = re.sub(r"^Q\d+_[A-Za-z]+_", "", col_name).strip()

    # Общая очистка от номера вопроса в начале (если остался)
    match_q_num = re.match(r"^\s*\d+\.?\s*[:?]?\s*(.*)", cleaned)
    if match_q_num:
        cleaned = match_q_num.group(1).strip()

    # Замены для краткости
    replacements = {
        "Работаю на себя / самозанятый / предприниматель": "Самозанятый/ИП",
        "Получаю образование": "Студент (Род деят.)",
        "Продажи и клиентский сервис": "Продажи/Сервис",
        "Медицина и здравоохранение": "Медицина",
        "Образование и наука": "Наука/Образование",
        "Государственная служба": "Госслужба",
        "Производство и инженерия": "Произв./Инж.",
        "Строительство и недвижимость": "Стр-во/Недв.",
        "Финансы и бухгалтерия": "Финансы/Бухг.",
        "С какой профессиональной областью вы себя идентифицируете?": "Проф.область:",
        "Укажите род вашей деятельности.": "Род деят.:",
        "Что повлияло на формирование ваших представлений об ИИ?": "",
        "(укажите факультет)": "",
        "(укажите)": "",
        "Сколько вам полных лет?": "Возраст",
        "Укажите ваш уровень дохода в месяц (после вычета налогов).": "Доход",
        "Насколько ваше финансовое положение позволяет вам приобретать следующее": "Фин.Благосост.",
        # Дополнительные очистки для длинных названий из гридов/MC
        "оцените насколько вы согласны или не согласны со следующими утверждениями": "",
        "оцените насколько вы бы доверили ему следующие задачи": "Доверие задаче:",
        "представьте что ии может предложить вам персонализированную помощь и поддержку оцените насколько вы будете готовы поделиться с ии следующей информацией": "Готовность делиться:",
        "как изменится ваше доверие к ии в следующих случаях": "Доверие изменится если:",
        "оцените насколько вас тревожат следующие потенциальные проблемы в работе ии": "Тревожат проблемы:",
        "насколько для вас важны следующие аспекты идеального взаимодействия с ии": "Важность аспекта ИИ:",
        "представьте что на работе вам необходимо взаимодействовать с ии оцените насколько указанные факторы могут усилить вашу мотивацию использовать ии": "Мотивация к ИИ:",
        "_": " ",  # Заменим оставшиеся '_' на пробелы для читаемости
    }
    # Применяем замены
    # V38: Приводим к нижнему регистру ДО замен для надежности
    temp_cleaned = cleaned.lower()
    for old, new in replacements.items():
        temp_cleaned = temp_cleaned.replace(old.lower(), new).strip()

    # Убираем лишние пробелы   cleaned = re.sub(r'\s+', ' ', temp_cleaned).strip().capitalize() # Капитализируем начало

    # Обрезка по длине
    key_max_len = 100
    current_max_len = key_max_len if is_key else max_len
    if len(cleaned) > current_max_len:
        cleaned = cleaned[: current_max_len - 3] + "..."

    # Возвращаем очищенное имя или исходное (обрезанное), если очистка не удалась
    return cleaned if cleaned else col_name[:max_len]


# --- Функция интерпретации MWU (v2, без изменений логики) ---
def interpret_mwu_result_v2(mwu_df, desc_stats, portrait_col_name):
    # ... (код interpret_mwu_result_v2 как в оригинале) ...
    if mwu_df is None or mwu_df.empty or "p-val" not in mwu_df.columns:
        return None
    p_value = mwu_df["p-val"].iloc[0]
    if p_value >= 0.05:
        return None
    try:
        # V38: Улучшаем обработку случая, когда одна из групп пуста или имеет только NaN
        if 0 not in desc_stats.index or 1 not in desc_stats.index:
            logging.debug(
                f"MWU interp skip for '{portrait_col_name}': missing group stats."
            )
            return None

        mean_group0 = (
            desc_stats.loc[0, "mean"] if "mean" in desc_stats.columns else np.nan
        )
        mean_group1 = (
            desc_stats.loc[1, "mean"] if "mean" in desc_stats.columns else np.nan
        )
        median_group0 = (
            desc_stats.loc[0, "median"] if "median" in desc_stats.columns else np.nan
        )
        median_group1 = (
            desc_stats.loc[1, "median"] if "median" in desc_stats.columns else np.nan
        )

        # Используем медианы для порядковых, средние для бинарных/числовых
        val0 = (
            median_group0
            if portrait_col_name
            in [
                "Q2_AgeGroup",
                "Q11_IncomeLevel",
                "Q12_WellbeingLevel",
                "Q7_EducationLevel",
            ]
            else mean_group0
        )
        val1 = (
            median_group1
            if portrait_col_name
            in [
                "Q2_AgeGroup",
                "Q11_IncomeLevel",
                "Q12_WellbeingLevel",
                "Q7_EducationLevel",
            ]
            else mean_group1
        )

        if pd.isna(val0) or pd.isna(val1):
            logging.debug(
                f"MWU interp skip for '{portrait_col_name}': NaN in compared values."
            )
            return None

        clean_name_desc = clean_col_name_v2(portrait_col_name, max_len=40, is_key=False)
        # V38: Повысим пороги для большей уверенности в значимости различий
        diff_threshold_ord = 0.15  # Для порядковых/числовых
        diff_threshold_bin = 0.08  # Для бинарных

        # Определяем тип переменной для выбора порога и формулировки
        is_binary_likely = (
            "std" in desc_stats.columns
            and desc_stats["std"].max() < 0.51
            and val0 >= 0
            and val1 >= 0
            and val0 <= 1
            and val1 <= 1
        ) or portrait_col_name.startswith(
            (
                "Q8_",
                "Q10_",
                "Q19_",
                "Q21_",
                "Q23_",
                "Q25_",
                "Q27_",
                "Q30_",
                "Q32_",
                "Q33_",
                "Q34_",
                "Q35_",
                "Q43_",
                "Q51_",
                "Q61_",
                "Q70_",
                "Q71_",
            )
        )

        current_threshold = (
            diff_threshold_bin if is_binary_likely else diff_threshold_ord
        )

        if val1 > val0 + current_threshold:
            if portrait_col_name == "Q2_AgeGroup":
                return f"Старше ({clean_name_desc})"
            elif portrait_col_name == "Q11_IncomeLevel":
                return f"Выше доход ({clean_name_desc})"
            elif portrait_col_name == "Q12_WellbeingLevel":
                return f"Выше фин.благ. ({clean_name_desc})"
            elif portrait_col_name == "Q7_EducationLevel":
                return f"Выше образ. ({clean_name_desc})"
            elif is_binary_likely:
                return f'Чаще "{clean_name_desc}"'
            else:
                return f'Выше "{clean_name_desc}"'
        elif val1 < val0 - current_threshold:
            if portrait_col_name == "Q2_AgeGroup":
                return f"Моложе ({clean_name_desc})"
            elif portrait_col_name == "Q11_IncomeLevel":
                return f"Ниже доход ({clean_name_desc})"
            elif portrait_col_name == "Q12_WellbeingLevel":
                return f"Ниже фин.благ. ({clean_name_desc})"
            elif portrait_col_name == "Q7_EducationLevel":
                return f"Ниже образ. ({clean_name_desc})"
            elif is_binary_likely:
                return f'Реже "{clean_name_desc}"'
            else:
                return f'Ниже "{clean_name_desc}"'

    except Exception as e:
        logging.debug(f"Ошибка интерпретации MWU для '{portrait_col_name}': {e}")
    return None


# --- Функция интерпретации Корреляции (v2, улучшен порог) ---
def interpret_correlation_result(
    r, p, descriptor_col_name, corr_threshold=0.20
):  # Порог увеличен до 0.20
    if pd.isna(p) or p >= 0.05 or pd.isna(r) or abs(r) < corr_threshold:
        return None
    clean_name_desc = clean_col_name_v2(descriptor_col_name, max_len=40, is_key=False)
    strength = "Слаб." if abs(r) < 0.3 else ("Сред." if abs(r) < 0.5 else "Сильн.")
    direction = "Положит." if r > 0 else "Отрицат."
    return f"{direction} {strength} связь с '{clean_name_desc}' (r={r:.2f})"


# --- Функция интерпретации Хи-квадрат (v2, проверяет размер эффекта) ---
def interpret_chi2_result(
    chi2_test_result, descriptor_col_name, cramer_v_threshold=0.15
):  # Добавлен порог для V Крамера
    if chi2_test_result is None or len(chi2_test_result) < 3:
        return None  # Ожидаем 3 таблицы от pingouin
    stats_df = chi2_test_result[1]
    expected_df = chi2_test_result[2]
    if (
        stats_df.empty
        or "p-val" not in stats_df.columns
        or "cramer" not in stats_df.columns
    ):
        return None

    # Проверка на слишком низкие ожидаемые частоты (может исказить хи-квадрат)
    if expected_df.min().min() < 5:
        logging.debug(
            f"Chi2 interp skip for '{descriptor_col_name}': Expected freq < 5."
        )
        return None

    p_value = stats_df["p-val"].iloc[0]
    cramer_v = stats_df["cramer"].iloc[0]
    if p_value < 0.05 and cramer_v >= cramer_v_threshold:
        clean_name_desc = clean_col_name_v2(
            descriptor_col_name, max_len=40, is_key=False
        )
        return f"Связано с '{clean_name_desc}' (V={cramer_v:.2f})"
    return None


# --- Основная функция анализа (V38 - Адаптированная) ---
def generate_variable_portraits(df, df_name):
    """
    Генерирует портреты для переменных Q12+, анализируя их связь
    с дескрипторами Q1-Q11 для указанного датафрейма.
    Работает с финальными именами колонок из df_processed_sorted.
    """
    # Коммент для себя переделать Q12+ в Q13+, и прописать Q1-Q11 как дескрипторы(группирующие переменные)
    portraits = defaultdict(list)
    min_group_size_analysis = 15  # Увеличим мин. размер группы
    total_comparisons = 0
    significant_findings = 0

    if df is None or df.empty:
        logging.error(
            f"[{df_name}] DataFrame пуст или не предоставлен. Анализ невозможен."
        )
        return

    all_cols = df.columns.tolist()

    # --- Определяем колонки Дескрипторов (Q1-Q12) ---
    descriptor_cols_present = []
    try:
        # Паттерны для поиска дескрипторов по финальным именам
        # Q2-Q7, Q9, Q11 - Одиночные вопросы (ординальные/категориальные)
        # Q8, Q10 - Multi-choice группы
        desc_pattern = r"^Q(2|3|4|5|6|7|8|9|10|11|12)_"
        descriptor_cols_present = [
            col for col in all_cols if re.match(desc_pattern, col)
        ]

        # Убедимся, что participant_id и Duration_seconds не попали
        descriptor_cols_present = [
            c
            for c in descriptor_cols_present
            if c not in ["participant_id", "Duration_seconds"]
        ]

        # Исключим колонки UNPROCESSED_, если они есть
        descriptor_cols_present = [
            c for c in descriptor_cols_present if not c.startswith("UNPROCESSED_")
        ]

        logging.info(
            f"[{df_name}] Найдено {len(descriptor_cols_present)} дескрипторных колонок (Q1-Q11)."
        )
        # logging.debug(f"Дескрипторы: {descriptor_cols_present[:10]}...") # Для отладки
    except Exception as e:
        logging.error(f"[{df_name}] Ошибка при определении дескрипторных колонок: {e}")
        traceback.print_exc()
        return

    # --- Определяем Целевые колонки (Q12+) ---
    target_cols_present = []
    try:
        # Паттерн для поиска целевых переменных (Q12 и выше)
        # Q12_... Q13_... Q71_... etc.
        target_pattern = r"^Q(1[3-9]|[2-9]\d|\d{4,})_"
        target_cols_present = [col for col in all_cols if re.match(target_pattern, col)]

        # Исключим колонки UNPROCESSED_
        target_cols_present = [
            c for c in target_cols_present if not c.startswith("UNPROCESSED_")
        ]

        logging.info(
            f"[{df_name}] Найдено {len(target_cols_present)} целевых колонок (Q12+)."
        )
        # logging.debug(f"Целевые: {target_cols_present[:10]}...") # Для отладки
    except Exception as e:
        logging.error(f"[{df_name}] Ошибка при определении целевых колонок: {e}")
        traceback.print_exc()
        return

    # --- Проведение анализа ---
    if not target_cols_present or not descriptor_cols_present:
        logging.warning(
            f"[{df_name}] Недостаточно колонок для анализа (Целевых: {len(target_cols_present)}, Дескрипторов: {len(descriptor_cols_present)})."
        )
        return

    print(
        f"\n[{df_name}] Анализ портретов для {len(target_cols_present)} переменных Q13+ vs {len(descriptor_cols_present)} дескрипторов Q1-Q12..."
    )
    processed_targets = 0

    for target_col in target_cols_present:
        processed_targets += 1
        if processed_targets % 50 == 0:
            print(
                f" [{df_name}] Обработано {processed_targets}/{len(target_cols_present)} целевых..."
            )

        try:  # Обернем обработку одной целевой переменной
            target_data = df[target_col].dropna()
            if target_data.empty or target_data.nunique() < 2:
                continue

            # Определяем тип целевой переменной
            is_binary_target = (
                target_data.nunique() == 2 and target_data.isin([0, 1]).all()
            )
            # V38: Уточняем определение числовой шкалы (больше уникальных значений)
            is_numeric_scale_target = (
                pd.api.types.is_numeric_dtype(df[target_col])
                and target_data.nunique() > 5
            )

            current_target_descriptors = (
                {}
            )  # Дескрипторы для текущей целевой переменной

            for desc_col in descriptor_cols_present:
                if desc_col == target_col:
                    continue  # Не сравниваем переменную с собой

                total_comparisons += 1

                try:  # Обернем обработку одной пары
                    temp_df_pair = df[[target_col, desc_col]].dropna()
                    if len(temp_df_pair) < min_group_size_analysis * 2:
                        continue  # Нужно достаточно данных

                    descriptor = None
                    p_val = 1.0
                    effect_size = 0.0

                    # --- Логика выбора теста ---
                    desc_dtype = df[desc_col].dtype
                    desc_nunique = temp_df_pair[desc_col].nunique()

                    # 1. Целевая БИНАРНАЯ (0/1)
                    if is_binary_target:
                        # Проверка размера групп
                        group_counts_pair = temp_df_pair[target_col].value_counts()
                        if (
                            0 not in group_counts_pair
                            or 1 not in group_counts_pair
                            or group_counts_pair[0] < min_group_size_analysis
                            or group_counts_pair[1] < min_group_size_analysis
                        ):
                            continue

                        # 1а. Дескриптор ЧИСЛОВОЙ -> MWU
                        if (
                            pd.api.types.is_numeric_dtype(desc_dtype)
                            and desc_nunique > 1
                        ):
                            group0 = temp_df_pair[temp_df_pair[target_col] == 0][
                                desc_col
                            ]
                            group1 = temp_df_pair[temp_df_pair[target_col] == 1][
                                desc_col
                            ]
                            if group0.nunique() < 2 or group1.nunique() < 2:
                                continue  # Нужно разнообразие в группах
                            mwu_res = pg.mwu(group0, group1, alternative="two-sided")
                            if not mwu_res.empty:
                                p_val = mwu_res["p-val"].iloc[0]
                                if p_val < 0.05:
                                    desc_stats = temp_df_pair.groupby(target_col)[
                                        desc_col
                                    ].agg(["mean", "median", "std", "count"])
                                    descriptor = interpret_mwu_result_v2(
                                        mwu_res, desc_stats, desc_col
                                    )

                        # 1б. Дескриптор КАТЕГОРИАЛЬНЫЙ (включая бинарные дескрипторы) -> Chi2
                        elif desc_nunique > 1:  # Не числовой и больше 1 категории
                            try:
                                chi2_res = pg.chi2_independence(
                                    data=temp_df_pair,
                                    x=desc_col,
                                    y=target_col,
                                    correction=False,
                                )  # correction=False т.к. размеры групп уже проверены
                                if (
                                    chi2_res and len(chi2_res) >= 3
                                ):  # Ожидаем 3 df от pingouin
                                    descriptor = interpret_chi2_result(
                                        chi2_res, desc_col
                                    )
                                    if descriptor:
                                        p_val = chi2_res[1]["p-val"].iloc[0]
                                        effect_size = chi2_res[1]["cramer"].iloc[0]
                            except (
                                ValueError
                            ) as ve:  # Ловим ошибку, если все значения в одной ячейке таблицы сопряженности
                                logging.debug(
                                    f"Chi2 error for {target_col} vs {desc_col}: {ve}"
                                )
                            except Exception as chi_e:
                                logging.warning(
                                    f"Unexpected Chi2 error for {target_col} vs {desc_col}: {chi_e}"
                                )
                                traceback.print_exc()

                    # 2. Целевая ЧИСЛОВАЯ/ШКАЛА
                    elif is_numeric_scale_target:
                        # 2а. Дескриптор ЧИСЛОВОЙ -> Корреляция (Spearman)
                        if (
                            pd.api.types.is_numeric_dtype(desc_dtype)
                            and desc_nunique > 1
                        ):
                            corr_res = pg.corr(
                                temp_df_pair[target_col],
                                temp_df_pair[desc_col],
                                method="spearman",
                            )
                            if not corr_res.empty:
                                r = corr_res["r"].iloc[0]
                                p = corr_res["p-val"].iloc[0]
                                descriptor = interpret_correlation_result(
                                    r, p, desc_col
                                )
                                if descriptor:
                                    p_val = p
                                    effect_size = abs(r)

                        # 2б. Дескриптор КАТЕГОРИАЛЬНЫЙ (2+ группы) -> Kruskal-Wallis
                        elif desc_nunique > 1:
                            # Проверка размера групп для Крускала
                            group_sizes = temp_df_pair.groupby(desc_col)[
                                target_col
                            ].count()
                            if (group_sizes >= min_group_size_analysis).all():
                                kruskal_res = pg.kruskal(
                                    data=temp_df_pair, dv=target_col, between=desc_col
                                )
                                if not kruskal_res.empty:
                                    p = kruskal_res["p-unc"].iloc[0]
                                    if p < 0.05:
                                        # Рассчитаем эпсилон-квадрат как меру эффекта
                                        H = kruskal_res["H"].iloc[0]
                                        N = len(temp_df_pair)
                                        eps_sq = H / (
                                            (N**2 - 1) / (N + 1)
                                        )  # Формула Epsilon-squared
                                        if (
                                            eps_sq >= 0.04
                                        ):  # Порог для размера эффекта (средний эффект ~0.04)
                                            clean_name_desc = clean_col_name_v2(
                                                desc_col, max_len=40, is_key=False
                                            )
                                            descriptor = f"Различается по '{clean_name_desc}' (ε²={eps_sq:.2f})"
                                            p_val = p
                                            effect_size = eps_sq
                            else:
                                logging.debug(
                                    f"Kruskal skip for {target_col} vs {desc_col}: small group size ({group_sizes.min()})"
                                )

                    # --- Сохраняем лучший дескриптор для этой целевой ---
                    if descriptor:
                        significant_findings += 1
                        # Сохраняем, если дескриптора еще нет ИЛИ новый результат более значим (меньше p ИЛИ больше эффект при том же p)
                        current_p = current_target_descriptors.get(desc_col, {}).get(
                            "p", 1.1
                        )
                        current_effect = current_target_descriptors.get(
                            desc_col, {}
                        ).get("effect", -0.1)
                        if p_val < current_p or (
                            p_val == current_p and effect_size > current_effect
                        ):
                            current_target_descriptors[desc_col] = {
                                "desc": descriptor,
                                "p": p_val,
                                "effect": effect_size,
                            }

                except Exception as e_pair:
                    logging.warning(
                        f"Ошибка анализа пары {target_col} vs {desc_col}: {e_pair}"
                    )
                    # traceback.print_exc() # Можно раскомментить для детальной отладки
                    pass  # Продолжаем со следующей парой

            # --- Сохраняем результаты для текущей целевой переменной ---
            if current_target_descriptors:
                # V38: Используем clean_col_name_v2 для ключа словаря portraits
                target_clean_key_name = clean_col_name_v2(
                    target_col, max_len=80, is_key=True
                )  # Увеличил max_len для ключа
                # Добавляем дескрипторы в список для этого ключа
                portraits[target_clean_key_name].extend(
                    current_target_descriptors.values()
                )

        except Exception as e_target:
            logging.error(
                f"Критическая ошибка при обработке целевой переменной {target_col}: {e_target}"
            )
            traceback.print_exc()
            pass  # Продолжаем со следующей целевой


# Функция основной обработки
def apply_processing_map_v37_concat(df, processing_map):
    logging.info("Запуск финальной обработки по карте (V37 - concat)...")
    temp_series_dict = defaultdict(list)
    processed_original_cols = set()
    all_new_col_names = set()
    final_data = {}
    id_col_original_found = False
    # Шаг 1: Обработка ID
    for key, config in processing_map.items():
        if isinstance(config, dict) and config.get("action") == "keep_as_id":
            if key in df.columns:
                final_data[config["new_name"]] = df[key].copy()
                processed_original_cols.add(key)
                all_new_col_names.add(config["new_name"])
                logging.info(f"Обработан ID: '{key}' -> '{config['new_name']}'")
                id_col_original_found = True
                break
            else:
                logging.error(f"Столбец ID '{key}' не найден!")
                return None, None
    if not id_col_original_found:
        logging.error("Нет 'keep_as_id' в карте!")
        return None, None

    # Шаг 2: Обработка остальных колонок
    logging.info("Шаг 2: Обработка оригинальных колонок...")
    for key_pattern, config in processing_map.items():
        if not isinstance(config, dict):
            continue  # Пропуск не-словарей в карте
        action = config.get("action")
        if not action or action in ["keep_as_id", "drop", "rank"]:
            continue

        is_mc_action = action == "mc_group"
        is_grid_action = action.endswith("_grid")
        new_name_base = (
            config.get("base_name") if is_mc_action else config.get("new_name")
        )
        params = config.get("params", {})
        # Поиск колонок
        try:
            matched_cols = []
            if key_pattern in df.columns and key_pattern not in processed_original_cols:
                matched_cols = [key_pattern]
            else:
                pattern_regex = rf"^\s*\d*\s*[\.:\)]?\s*{re.escape(key_pattern)}"
                matched_cols = [
                    col
                    for col in df.columns
                    if re.match(pattern_regex, str(col).strip(), re.IGNORECASE)
                    and col not in processed_original_cols
                ]
        except re.error as e:
            logging.error(f"Regex error for '{key_pattern[:60]}...': {e}. Skip.")
            continue
        # Пропуск альтернативных ключей
        if not matched_cols:
            is_alternative = False
            if "." in key_pattern and key_pattern[0].isdigit():
                base_num = key_pattern.split(".")[0]
                alternatives = {
                    k
                    for k, v in processing_map.items()
                    if isinstance(v, dict) and k.startswith(base_num + ".")
                }
                if len(alternatives) > 1 and key_pattern != min(alternatives, key=len):
                    is_alternative = True
            if not is_alternative:
                logging.debug(
                    f"Паттерн '{key_pattern[:60]}...' не найден, пропуск '{action}'."
                )
            continue
        # Обработка найденных колонок
        logging.debug(
            f"Обработка '{key_pattern[:60]}...' ({action}), найдено: {len(matched_cols)}."
        )
        for original_col in matched_cols:
            if original_col in processed_original_cols:
                continue
            item_suffix = ""
            if is_grid_action or is_mc_action:
                item_suffix_extracted = _generate_col_suffix(original_col, key_pattern)
                item_suffix = (
                    f"_{item_suffix_extracted}"
                    if item_suffix_extracted
                    else f"_item{matched_cols.index(original_col) + 1}"
                )
            final_col_name = f"{new_name_base}{item_suffix}" if new_name_base else None
            if not final_col_name:
                logging.error(f"Не генер. имя для '{original_col}'")
                processed_original_cols.add(original_col)
                continue
            # Обработка значений
            output_series = None
            try:
                target_series = df[original_col]
                if action in ["numeric", "numeric_grid"]:
                    output_series = pd.to_numeric(target_series, errors="coerce")
                elif action in ["map", "map_grid"]:
                    map_to_use = params.get("map")
                    if map_to_use and isinstance(map_to_use, dict):
                        clean_series = target_series.apply(_clean_string_for_mapping)
                        output_series = clean_series.map(map_to_use)
                    else:
                        logging.error(f"Нет карты для '{action}' / '{original_col}'")
                elif action == "timedelta_to_seconds":

                    def to_seconds(td):
                        if pd.isna(td):
                            return np.nan
                        try:
                            if isinstance(td, datetime.timedelta):
                                return td.total_seconds()
                            if isinstance(td, (int, float)) and td < 2:
                                return td * 24 * 3600
                            if isinstance(td, datetime.time):
                                return (
                                    td.hour * 3600
                                    + td.minute * 60
                                    + td.second
                                    + td.microsecond / 1e6
                                )
                            if isinstance(td, str) and ":" in td:
                                parts = td.split(":")
                                sec = 0.0
                                try:
                                    if len(parts) == 3:
                                        sec = (
                                            float(parts[0]) * 3600
                                            + float(parts[1]) * 60
                                            + float(parts[2])
                                        )
                                    elif len(parts) == 2:
                                        sec = float(parts[0]) * 60 + float(parts[1])
                                    return sec
                                except ValueError:
                                    return np.nan
                            return pd.to_numeric(td, errors="coerce")
                        except Exception:
                            return np.nan

                    output_series = target_series.apply(to_seconds)
                elif action == "mc_group":
                    output_series = target_series.notna().astype(int)
                # Сохранение результата
                if output_series is not None and output_series.notna().any():
                    counter = 1
                    temp_final_name = final_col_name
                    original_final_name = final_col_name
                    while temp_final_name in temp_series_dict:
                        temp_final_name = f"{original_final_name}_DUPL{counter}"
                        counter += 1
                    if temp_final_name != original_final_name:
                        logging.warning(
                            f"Дубль имени '{original_final_name}', новое: '{temp_final_name}'"
                        )
                    final_col_name = temp_final_name
                    temp_series_dict[final_col_name].append(output_series)
                    all_new_col_names.add(final_col_name)
                processed_original_cols.add(original_col)
            except Exception as e_action:
                logging.error(
                    f"Ошибка '{action}' для '{original_col}' -> '{final_col_name}': {e_action}"
                )
                traceback.print_exc()
                processed_original_cols.add(original_col)

    # Шаг 3: Объединение серий
    logging.info("Шаг 3: Объединение и финализация колонок...")
    processed_final_names = set(final_data.keys())
    for final_name, series_list in temp_series_dict.items():
        if final_name in processed_final_names:
            continue
        combined_series = None
        if len(series_list) == 1:
            combined_series = series_list[0]
        elif len(series_list) > 1:
            combined_series = series_list[0].copy()  # Явно копируем
            for i in range(1, len(series_list)):
                combined_series = combined_series.combine_first(series_list[i])
            logging.info(
                f"  Колонка '{final_name}': объединено {len(series_list)} серии."
            )
        else:
            continue
        # Конвертация типов
        if combined_series is not None:
            is_mc_col = any(
                isinstance(cfg, dict)
                and cfg.get("action") == "mc_group"
                and cfg.get("base_name")
                and final_name.startswith(cfg["base_name"])
                for cfg in processing_map.values()
            )
            if is_mc_col:
                final_series = combined_series.fillna(0).astype(int)
            else:
                final_series = pd.to_numeric(combined_series, errors="coerce")
                if not (
                    pd.api.types.is_float_dtype(final_series.dtype)
                    or pd.api.types.is_integer_dtype(final_series.dtype)
                ):
                    logging.error(
                        f"  -> !!! Тип '{final_name}' НЕ ЧИСЛОВОЙ ({final_series.dtype})."
                    )
                # original_nan = combined_series.isna().sum(); final_nan = final_series.isna().sum()
                # if final_nan > original_nan: logging.warning(f"     -> В '{final_name}' стало больше NaN ({final_nan} > {original_nan}).")
            final_data[final_name] = final_series
            processed_final_names.add(final_name)

    # Шаг 4: Добавление необработанных
    original_cols_in_df = set(df.columns)
    explicit_drops = {
        key
        for key, config in processing_map.items()
        if isinstance(config, dict) and config.get("action") == "drop"
    }
    rank_keys_ignored = {
        k
        for k, v in processing_map.items()
        if isinstance(v, dict) and v.get("action") == "rank"
    }
    # Находим исходный ключ ID для добавления в accounted_for
    id_original_key = id_col  # По умолчанию
    for k, cfg in processing_map.items():
        if isinstance(cfg, dict) and cfg.get("action") == "keep_as_id":
            id_original_key = k
            break
    accounted_for = (
        processed_original_cols | explicit_drops | rank_keys_ignored | {id_original_key}
    )
    unaccounted_cols = original_cols_in_df - accounted_for
    if unaccounted_cols:
        logging.warning(f"!!! {len(unaccounted_cols)} столбцов не обработаны/удалены.")
        logging.warning(
            f"    Примеры: {sorted(list(unaccounted_cols))[:5]}{'...' if len(unaccounted_cols) > 5 else ''}"
        )
        logging.warning("    Добавляем с префиксом UNPROCESSED_.")
        for col in sorted(list(unaccounted_cols)):
            if col in df:
                new_unprocessed_name = f"UNPROCESSED_{col}"
                counter = 1
                original_unprocessed_name = new_unprocessed_name
                while new_unprocessed_name in final_data:
                    new_unprocessed_name = f"{original_unprocessed_name}_{counter}"
                    counter += 1
                final_data[new_unprocessed_name] = df[col].copy()
    else:
        logging.info("Все оригинальные колонки обработаны/удалены.")

    # Шаг 5: Создание DataFrame
    try:
        df_final = pd.DataFrame(final_data)
        # df_final = df_final.copy() # Копия создается в конце блока try/except
        logging.info(f"Создан итоговый DataFrame с {len(df_final.columns)} колонками.")
        return df_final.copy(), processed_original_cols  # Возвращаем копию
    except Exception as e_create_df:
        logging.error(f"Ошибка создания финального DataFrame: {e_create_df}")
        traceback.print_exc()
        return None, None


# Функция динамической сортировки
def sort_columns_dynamically(df):
    if df is None or df.empty:
        return df
    cols = df.columns.tolist()
    # Фиксируем начало
    fixed_start = []
    if final_participant_id_col in cols:
        fixed_start.append(final_participant_id_col)
        cols.remove(final_participant_id_col)
    else:
        logging.error(f"ID column '{final_participant_id_col}' not found for sorting.")
    if "Duration_seconds" in cols:
        fixed_start.append("Duration_seconds")
        cols.remove("Duration_seconds")

    # Ключ сортировки
    def get_sort_key(col_name):
        if col_name.startswith("UNPROCESSED_"):
            return (float("inf"), 2, col_name)  # Необработанные в конец
        match = re.match(r"Q(\d+)", col_name, re.IGNORECASE)  # Ищем номер вопроса QXX_
        if match:
            q_num = int(match.group(1))
            suffix = col_name[len(match.group(0)) :].lstrip(
                "_"
            )  # Остаток имени после QXX_
            is_rank = (
                "rank" in col_name.lower()
            )  # Признак Rank (не используется, но оставлен)
            is_mc_grid = "_" in suffix  # Признак сетки/MC (есть еще суффикс)
            type_priority = (
                2 if is_rank else (1 if is_mc_grid else 0)
            )  # Ранки, потом сетки/MC, потом одиночные
            return (q_num, type_priority, suffix)  # Сортируем по номеру, типу, суффиксу
        else:
            return (float("inf"), 1, col_name)  # Все остальное - в конец

    # Сортируем оставшиеся колонки
    sorted_cols = sorted(cols, key=get_sort_key)
    final_order = fixed_start + sorted_cols
    logging.info(
        f"Динамически определенный порядок колонок ({len(final_order)}): {final_order[:15]}..."
    )

    # Проверка на потерю/добавление колонок при сортировке
    final_order_set = set(final_order)
    original_cols_set = set(df.columns)
    if final_order_set != original_cols_set:
        missing_in_final = original_cols_set - final_order_set
        extra_in_final = final_order_set - original_cols_set
        logging.error("!!! ОШИБКА СОРТИРОВКИ: Несоответствие колонок!")
        if missing_in_final:
            logging.error(f"  Отсутствуют в final_order: {missing_in_final}")
        if extra_in_final:
            logging.error(f"  Лишние в final_order: {extra_in_final}")
        # Пытаемся исправить, добавляя недостающие в конец
        final_order.extend(list(missing_in_final))
        final_order = [
            c for c in final_order if c in original_cols_set
        ]  # Убираем лишние
        logging.warning("  Порядок колонок скорректирован.")
    # Возвращаем DataFrame с новым порядком колонок
    return df[final_order]


# --- 4. Карты Значений ---
# (Здесь все определения age_map, gender_map, ..., create_cleaned_map как в V37)
age_map = create_cleaned_map(
    {
        "до 18 лет": 0.0,
        "18 - 24 года": 1.0,
        "25 - 34 года": 2.0,
        "25 - 34 лет": 2.0,
        "35 - 44 года": 3.0,
        "35 - 44 лет": 3.0,
        "45 - 64 года": 4.0,
        "45 - 64 лет": 4.0,
        "65 лет и старше": 5.0,
    }
)
gender_map = create_cleaned_map({"мужской": 0.0, "женский": 1.0})
location_map = create_cleaned_map(
    {
        "москва, санкт-петербург": 1.0,
        "город-миллионник, кроме москвы и санкт-петербурга": 2.0,
        "город с населением от 50 тыс до 1 миллиона человек": 3.0,
        "город с населением от 50 тыс. до 1 млн чел.": 3.0,
        "населенный пункт с населением до 50 тыс человек": 4.0,
    }
)
marital_map = create_cleaned_map(
    {
        "состою в браке / гражданском браке / отношениях": 1.0,
        "холост/разведен/вдовствую": 0.0,
    }
)
children_map = create_cleaned_map(
    {"нет": 0.0, "один ребенок": 1.0, "двое детей": 2.0, "трое и более детей": 3.0}
)
position_map_simple = create_cleaned_map(
    {
        "стажер": 1.0,
        "специалист": 2.0,
        "менеджер": 3.0,
        "руководитель отдела": 4.0,
        "директор / исполнительный директор": 5.0,
        "предприниматель / владелец бизнеса": 6.0,
        "преподаватель / научный сотрудник": 7.0,
        "студент": 8.0,
    }
)
income_map = create_cleaned_map(
    {
        "до 50 000 р": 1.0,
        "50 001 - 100 000 р": 2.0,
        "100 001 - 150 000 р": 3.0,
        "150 001 - 200 000 р": 4.0,
        "200 001 - 400 000 р": 5.0,
        "400 001 р и выше": 6.0,
        "400 001 р. и выше": 6.0,
    }
)
wellbeing_map = create_cleaned_map(
    {
        "денег не хватает даже на еду": 1.0,
        "на еду хватает, но покупка одежды вызывает затруднение": 2.0,
        "на одежду хватает, но покупка бытовой техники вызывает затруднение": 3.0,
        "на бытовую технику хватает, но покупка автомобиля вызывает затруднения": 4.0,
        "на автомобиль хватает": 5.0,
    }
)
education_map = create_cleaned_map(
    {"среднее общее и ниже": 1.0, "среднее специальное": 2.0, "высшее": 3.0}
)
freq_map_1_5 = create_cleaned_map(
    {"никогда": 1.0, "редко": 2.0, "иногда": 3.0, "часто": 4.0, "всегда": 5.0}
)
freq_map_1_5_reversed = create_cleaned_map(
    {"всегда": 5.0, "часто": 4.0, "иногда": 3.0, "редко": 2.0, "никогда": 1.0}
)  # Важно: Никогда = 1
freq_map_ai_usage = create_cleaned_map(
    {
        "никогда": 1.0,
        "редко (1–2 раза в месяц)": 2.0,
        "редко (раз в несколько месяцев)": 2.0,
        "иногда (1–2 раза в неделю)": 3.0,
        "иногда (несколько раз в месяц)": 3.0,
        "часто (несколько раз в неделю)": 4.0,
        "очень часто (ежедневно)": 5.0,
    }
)
freq_map_veryoften = create_cleaned_map(
    {"никогда": 1.0, "редко": 2.0, "иногда": 3.0, "часто": 4.0, "очень часто": 5.0}
)
freq_map_comm_1_5 = create_cleaned_map(
    {
        "никогда": 1.0,
        "редко (1–2 раза в месяц)": 2.0,
        "иногда (1–2 раза в неделю)": 3.0,
        "часто (несколько раз в неделю)": 4.0,
        "очень часто (ежедневно)": 5.0,
    }
)
understanding_map = create_cleaned_map(
    {
        "совсем не понимаю, не представляю, как он работает": 1.0,
        "примерно представляю - слышал про chatgpt, нейросети, алгоритмы, но не знаю, как они устроены": 2.0,
        "обладаю базовыми знаниями - понимаю основные принципы, но не могу объяснить детали": 3.0,
        "обладаю базовыми знаниями - понимаю основные принципы , но не могу объяснить детали": 3.0,
        "хорошо разбираюсь - знаю, как работают модели машинного обучения, какие данные им нужны, могу объяснить это другим": 4.0,
        "глубоко понимаю - имею профессиональные знания, разбираюсь в алгоритмах, методах обучения и разработке ии": 5.0,
    }
)
yes_no_rather_map = create_cleaned_map({"скорее нет": 0.0, "скорее да": 1.0})
confidence_map_5 = create_cleaned_map(
    {
        "совершенно неуверенно": 1.0,
        "скорее неуверенно": 2.0,
        "нейтрально": 3.0,
        "скорее уверенно": 4.0,
        "абсолютно уверенно": 5.0,
    }
)
ai_balance_map = create_cleaned_map(
    {
        "вариант 1: человек - 90%, ии - 10%": 1.0,
        "вариант 2: человек - 70%, ии - 30%": 2.0,
        "вариант 3: человек - 50%, ии - 50%": 3.0,
        "вариант 4: человек - 30%, ии - 70%": 4.0,
        "вариант 5: человек - 10%, ии - 90%": 5.0,
    }
)
agreement_map_5_text = create_cleaned_map(
    {
        "полностью не согласен": 1.0,
        "скорее не согласен": 2.0,
        "нейтрален": 3.0,
        "скорее согласен": 4.0,
        "полностью согласен": 5.0,
    }
)
future_work_map = create_cleaned_map(
    {
        "сделает работу сложнее, приведет к сокращению рабочих мест или создаст новые риски": 1.0,
        "скорее негативно повлияет, потребует сложной адаптации": 2.0,
        "существенных изменений не ожидаю": 3.0,
        "поможет автоматизировать рутинные задачи, повысит эффективность": 4.0,
        "откроет новые карьерные возможности, значительно улучшит условия труда": 5.0,
    }
)
future_life_map = create_cleaned_map(
    {
        "приведет к перегрузке технологиями, создаст больше проблем, чем удобств": 1.0,
        "может вызывать сложности, но в целом не критично": 2.0,
        "существенных изменений не ожидаю": 3.0,
        "облегчит некоторые повседневные задачи": 4.0,
        "существенно повысит комфорт, сделает жизнь удобнее и проще": 5.0,
    }
)
future_comm_map = create_cleaned_map(
    {
        "ухудшит коммуникацию, заменит живое общение технологиями": 1.0,
        "может усложнить взаимодействие": 2.0,
        "существенных изменений не ожидаю": 3.0,
        "упростит поиск нужных контактов, персонализирует общение": 4.0,
        "существенно расширит возможности общения, сделает его удобнее": 5.0,
    }
)
future_secure_map = create_cleaned_map(
    {
        "станет причиной кризиса, порождая киберугрозы, утечки данных, манипуляции информацией и т д": 1.0,
        "станет причиной кризиса, порождая киберугрозы, утечки данных, манипуляции информацией и т. д.": 1.0,
        "скорее негативно повлияет, создавая новые риски": 2.0,
        "существенных изменений в безопасности не ожидаю": 3.0,
        "поможет предотвращать угрозы, повысит защиту личных данных": 4.0,
        "существенно укрепит безопасность, создаст более стабильную среду": 5.0,
    }
)
future_learn_map = create_cleaned_map(
    {
        "ограничит оригинальность, снизит потребность в самостоятельном мышлении": 1.0,
        "может быть полезен, но не заменит традиционные методы": 2.0,
        "существенных изменений не ожидаю": 3.0,
        "откроет новые способы обучения, облегчит доступ к знаниям": 4.0,
        "существенно расширит творческие и образовательные возможности": 5.0,
    }
)
trust_map_l2 = create_cleaned_map(
    {
        "совершенно не доверяю": 1.0,
        "скорее не доверяю": 2.0,
        "нейтрально": 3.0,
        "скорее доверяю": 4.0,
        "полностью доверяю": 5.0,
    }
)
trust_change_map_l2 = create_cleaned_map(
    {
        "мое доверие к ии пропадет": 1.0,
        "мое доверие к ии снизится": 2.0,
        "мое доверие к ии не изменится": 3.0,
        "мое доверие к ии повысится": 4.0,
        "мое доверие к ии значительно возрастет": 5.0,
    }
)
yes_no_map_l2 = create_cleaned_map({"да": 1.0, "нет": 0.0})
ethical_risk_source_map_l2 = create_cleaned_map(
    {
        "не вижу этических рисков": 0.0,
        "с самим ии": 1.0,
        "с теми, кто его разрабатывает и применяет": 2.0,
        "и то, и другое": 3.0,
    }
)
preferred_ai_comm_map_l2 = create_cleaned_map(
    {
        "текстовый чат - общение посредством письменных сообщений": 1.0,
        "голосовое общение - диалог с ии посредством голосовых команд или аудиоконференций, аналогичных работе голосовых помощников": 2.0,
        "видео-консультация - общение с использованием видеосвязи, где присутствует визуальный контакт": 3.0,
        "все вместе - смешанный формат, сочетающий текст, аудио, видео и интерактивные элементы для более богатого опыта общения": 4.0,
    }
)
ai_decision_basis_map_l2 = create_cleaned_map(
    {"оптимизацией и максимизацией пользы": 0.0, "нормами человеческой морали": 1.0}
)
yes_no_map_l2_moral = create_cleaned_map(
    {"да": 1.0, "нет": 0.0, "скорее да": 1.0, "скорее нет": 0.0}
)


# --- 5. Карта Обработки (V37 + Дубликаты + ТОЧНЫЕ КЛЮЧИ + Комментарии) ---
processing_map = {
    # ==========================================================================
    # --- Технические / ID ---
    # ==========================================================================
    "participant_id": {"action": "keep_as_id", "new_name": "participant_id"},
    "Длительность": {"action": "timedelta_to_seconds", "new_name": "Duration_seconds"},
    # ==========================================================================
    # --- Блок 1: Демография (Q2-Q12) ---
    # (Присутствует в обеих анкетах, ключи должны быть унифицированы или дублироваться)
    # ==========================================================================
    "2. Сколько вам полных лет?": {
        "action": "map",
        "new_name": "Q2_AgeGroup",
        "params": {"map": age_map, "unmapped_to_nan": True},
    },
    "3. Укажите ваш пол.": {
        "action": "map",
        "new_name": "Q3_Gender",
        "params": {"map": gender_map, "unmapped_to_nan": True},
    },
    "4. В населенном пункте какого типа вы постоянно проживаете?": {
        "action": "map",
        "new_name": "Q4_LocationType",
        "params": {"map": location_map, "unmapped_to_nan": True},
    },
    "5. Укажите статус ваших отношений на данный момент?": {
        "action": "map",
        "new_name": "Q5_MaritalStatus",
        "params": {"map": marital_map, "unmapped_to_nan": True},
    },
    "6. У вас есть дети?": {
        "action": "map",
        "new_name": "Q6_ChildrenStatus",
        "params": {"map": children_map, "unmapped_to_nan": True},
    },
    "7. Укажите уровень вашего образования на данный момент.": {
        "action": "map",
        "new_name": "Q7_EducationLevel",
        "params": {"map": education_map, "unmapped_to_nan": True},
    },
    # Q8: Род деятельности (MC)
    "8. Укажите род вашей деятельности.": {
        "action": "mc_group",
        "base_name": "Q8_ActivityField",
    },
    # Q9: Должность (Map) - два варианта ключа
    "9. Укажите вашу текущую должность.": {
        "action": "map",
        "new_name": "Q9_Position",
        "params": {"map": position_map_simple, "unmapped_to_nan": True},
    },
    "9. Укажите вашу должность.": {
        "action": "map",
        "new_name": "Q9_Position",
        "params": {"map": position_map_simple, "unmapped_to_nan": True},
    },
    # Q10: Проф. область (MC) - два варианта ключа
    "10. С какой сферой связан ваш основной род деятельности?": {
        "action": "mc_group",
        "base_name": "Q10_ProfArea",
        "ignore_starts_with": ["Другое", "Студент"],
    },
    "10. С какой профессиональной областью вы себя идентифицируете?": {
        "action": "mc_group",
        "base_name": "Q10_ProfArea",
        "ignore_starts_with": ["Другое", "Студент"],
    },
    # Q11, Q12: Доход, Благосостояние (Map)
    "11. Укажите ваш примерный ежемесячный доход": {
        "action": "map",
        "new_name": "Q11_IncomeLevel",
        "params": {"map": income_map, "unmapped_to_nan": True},
    },
    "12. Как бы вы описали ваше материальное благосостояние?": {
        "action": "map",
        "new_name": "Q12_WellbeingLevel",
        "params": {"map": wellbeing_map, "unmapped_to_nan": True},
    },
    # ==========================================================================
    # --- Блок 1 Продолжение: Общие установки (Q13-Q16) ---
    # (Присутствует в обеих анкетах)
    # ==========================================================================
    # Q13: Установка на рост (GM - Growth Mindset) - Grid
    "13. Оцените, насколько вы согласны или не согласны со следующими утверждениями.": {
        "action": "numeric_grid",
        "new_name": "Q13_GM",
    },
    # Q14: Сопротивление изменениям (RC - Resistance to Change) - Grid
    "14. Оцените, насколько вы согласны или не согласны со следующими утверждениями.": {
        "action": "numeric_grid",
        "new_name": "Q14_RC",
    },  # Используется один и тот же ключ для обоих гридов Q13/Q14, но это OK, т.к. названия колонок будут отличаться
    # Q15: Частота проверки источников (Map) - два варианта ключа
    "15. Как часто вы проверяете достоверность новой информации по другим источникам?": {
        "action": "map",
        "new_name": "Q15_SourceVerifyFreq",
        "params": {"map": freq_map_1_5_reversed, "unmapped_to_nan": True},
    },
    "15. Как часто вы обращаетесь к дополнительным источникам для подтверждения достоверности информации?": {
        "action": "map",
        "new_name": "Q15_SourceVerifyFreq",
        "params": {"map": freq_map_1_5_reversed, "unmapped_to_nan": True},
    },
    # Q16: Успешность объяснения (Map)
    "16. Представьте, что вам нужно что-то объяснить человеку": {
        "action": "map",
        "new_name": "Q16_ExplainSuccessFreq",
        "params": {"map": freq_map_1_5, "unmapped_to_nan": True},
    },
    # ==========================================================================
    # --- Блок 2: Осведомленность и Опыт (Q18-Q30) ---
    # (Вопросы ТОЛЬКО из Анкеты 1)
    # ==========================================================================
    "18. Как бы вы оценили свое понимание того, как работает ИИ?": {
        "action": "map",
        "new_name": "Q18_AIUnderstanding",
        "params": {"map": understanding_map, "unmapped_to_nan": True},
    },
    "19. Какие технологии вы считаете искусственным интеллектом?": {
        "action": "mc_group",
        "base_name": "Q19_ConsideredAI",
        "ignore_starts_with": ["Другое"],
    },
    "21. С какими из этих ИИ технологий вы уже имели опыт взаимодействия?": {
        "action": "mc_group",
        "base_name": "Q21_Interact",
        "ignore_starts_with": ["Другое", "У меня не было"],
    },
    "22. Как часто вы используете технологии на базе ИИ?": {
        "action": "map",
        "new_name": "Q22_AITechUseFreq",
        "params": {"map": freq_map_ai_usage, "unmapped_to_nan": True},
    },
    "23. В каких областях вы чаще всего встречаетесь с ИИ технологиями?": {
        "action": "mc_group",
        "base_name": "Q23_EncounterArea",
        "ignore_starts_with": [
            "Другое",
            "Я не пользуюсь",
            "Не понимаю",
            "Я не сталкиваюсь",
        ],
    },
    "23. С какой целью вы обычно используете ИИ?": {
        "action": "mc_group",
        "base_name": "Q23_UsagePurpose",
    },  # Отличается от Q33
    "24. Как часто вы ищете информацию о новых технологиях в сфере ИИ?": {
        "action": "map",
        "new_name": "Q24_AISearchInfoFreq",
        "params": {"map": freq_map_veryoften, "unmapped_to_nan": True},
    },
    "25. Что повлияло на формирование ваших представлений об ИИ?": {
        "action": "mc_group",
        "base_name": "Q25_Influence",
    },
    "26. Сможете ли вы определить, что текст создан с помощью ИИ?": {
        "action": "map",
        "new_name": "Q26_AIDetectText",
        "params": {"map": yes_no_rather_map, "unmapped_to_nan": True},
    },
    "27. Укажите, по каким признакам вы понимаете, что текст создан с помощью ИИ.": {
        "action": "mc_group",
        "base_name": "Q27_Sign",
        "ignore_starts_with": ["Затрудняюсь ответить"],
    },
    "28. Вспомните ИИ, с которым вы чаще всего взаимодействовали. Без маски": {
        "action": "drop"
    },
    "29. Оцените, насколько вы согласны или не согласны со следующими утверждениями.": {
        "action": "numeric_grid",
        "new_name": "Q29_Eval",
    },  # Оценка взаимодействия
    "30. Имеете ли вы профессиональный опыт работы с ИИ?": {
        "action": "mc_group",
        "base_name": "Q30_ProfExp",
    },
    # ==========================================================================
    # --- Блок 3: Общее Отношение (Q32-Q51) ---
    # (Вопросы ТОЛЬКО из Анкеты 1)
    # ==========================================================================
    "32. Выберите метафору, которая лучше всего описывает": {
        "action": "mc_group",
        "base_name": "Q32_Metaphor",
    },
    "33. С какой целью вы обычно используете ИИ?": {
        "action": "mc_group",
        "base_name": "Q33_UsagePurpose",
    },  # Отличается от Q23
    "34. Какие из следующих тем вы не стали бы обсуждать с ИИ?": {
        "action": "mc_group",
        "base_name": "Q34_NotDiscuss",
        "ignore_starts_with": [
            "Я бы не обсуждал",
            "Я готов обсуждать",
            "Я бы не обсуждал(а)",
        ],
    },
    "35. Какие эмоции вызывают у вас мысли о внедрении ИИ": {
        "action": "mc_group",
        "base_name": "Q35_Emotion",
    },
    "36. Насколько уверенно вы чувствуете себя, используя технологии на базе ИИ?": {
        "action": "map",
        "new_name": "Q36_AIUseConfidence",
        "params": {"map": confidence_map_5, "unmapped_to_nan": True},
    },
    # Q37: Баланс Человек/ИИ - два варианта ключа
    "37. Какой баланс в работе между человеком и ИИ вы считаете оптимальным?": {
        "action": "map",
        "new_name": "Q37_AIBalancePref",
        "params": {"map": ai_balance_map, "unmapped_to_nan": True},
    },
    "24. Какой баланс в работе между человеком и ИИ вы считаете оптимальным?": {
        "action": "map",
        "new_name": "Q37_AIBalancePref",
        "params": {"map": ai_balance_map, "unmapped_to_nan": True},
    },
    # Q38: Антропоморфизм - Grid
    "38. Оцените, насколько вы согласны или не согласны со следующими утверждениями.": {
        "action": "numeric_grid",
        "new_name": "Q38_Anth",
    },
    "39. Как часто вы общаетесь с ИИ на личные темы": {
        "action": "map",
        "new_name": "Q39_AIPersonalCommFreq",
        "params": {"map": freq_map_comm_1_5, "unmapped_to_nan": True},
    },
    # Q40: Ранжирование ролей ИИ - Drop
    "40. Представьте, что у вас есть персональный ИИ-помощник...": {"action": "drop"},
    "40. Представьте, что у вас есть персональный ИИ-помощник. Персональный ИИ – это умный помощник, например, как «Алиса», но с более широкими возможностями. Распределите роли персонального ИИ по степени важности для вас.": {
        "action": "drop"
    },  # Точный ключ
    # Q41: Доверие к задачам - Grid
    "41. Представьте, что ИИ самостоятельно принимает решения в различных сферах.": {
        "action": "numeric_grid",
        "new_name": "Q41_Trust",
    },
    # Q42/Q69/Q44: Частота проверки инфо (объединено в Q42_InfoVerifyFreq)
    "42. Как часто вы обращаетесь к дополнительным источникам для подтверждения достоверности информации?": {
        "action": "map",
        "new_name": "Q42_InfoVerifyFreq",
        "params": {"map": freq_map_1_5_reversed, "unmapped_to_nan": True},
    },
    "44. Как часто вы анализируете информацию, используя дополнительные источники для подтверждения ее достоверности?": {
        "action": "map",
        "new_name": "Q42_InfoVerifyFreq",
        "params": {"map": freq_map_1_5_reversed, "unmapped_to_nan": True},
    },
    "69. Как часто вы обращаетесь к дополнительным источникам...": {
        "action": "map",
        "new_name": "Q42_InfoVerifyFreq",
        "params": {"map": freq_map_1_5_reversed, "unmapped_to_nan": True},
    },
    # Q43: Человек лучше ИИ (MC) - два варианта ключа
    "43. Как вы считаете, с какими задачами человек справляется лучше, чем ИИ?": {
        "action": "mc_group",
        "base_name": "Q43_HumanSup",
    },
    "42. Как вы считаете, с какими задачами человек справляется лучше, чем ИИ?": {
        "action": "mc_group",
        "base_name": "Q43_HumanSup",
    },
    # Q44: ИИ может критиковать (Map) - два варианта ключа
    "44. Согласны ли вы с утверждением, что ИИ может «проверять и критиковать» работу людей в их профессиональной области?": {
        "action": "map",
        "new_name": "Q44_AICanCritique",
        "params": {"map": agreement_map_5_text, "unmapped_to_nan": True},
    },
    "43. Согласны ли вы с утверждением, что ИИ может «проверять и критиковать» работу людей в их профессиональной области?": {
        "action": "map",
        "new_name": "Q44_AICanCritique",
        "params": {"map": agreement_map_5_text, "unmapped_to_nan": True},
    },
    # Q45-Q49: Будущее влияние (Map) - по два варианта ключа для каждого
    "45. Как, по вашему мнению, ИИ изменит вашу работу в будущем?": {
        "action": "map",
        "new_name": "Q45_FutureImpact_Work",
        "params": {"map": future_work_map, "unmapped_to_nan": True},
    },
    "44. Как, по вашему мнению, ИИ изменит вашу работу в будущем?": {
        "action": "map",
        "new_name": "Q45_FutureImpact_Work",
        "params": {"map": future_work_map, "unmapped_to_nan": True},
    },
    "46. Как ИИ может изменить вашу повседневную жизнь?": {
        "action": "map",
        "new_name": "Q46_FutureImpact_Life",
        "params": {"map": future_life_map, "unmapped_to_nan": True},
    },
    "45. Как ИИ может изменить вашу повседневную жизнь?": {
        "action": "map",
        "new_name": "Q46_FutureImpact_Life",
        "params": {"map": future_life_map, "unmapped_to_nan": True},
    },
    "47. Как, по вашему мнению, ИИ повлияет на общение между людьми?": {
        "action": "map",
        "new_name": "Q47_FutureImpact_Comm",
        "params": {"map": future_comm_map, "unmapped_to_nan": True},
    },
    "46. Как, по вашему мнению, ИИ повлияет на общение между людьми?": {
        "action": "map",
        "new_name": "Q47_FutureImpact_Comm",
        "params": {"map": future_comm_map, "unmapped_to_nan": True},
    },
    "48. Как ИИ повлияет на вашу защищенность и стабильность?": {
        "action": "map",
        "new_name": "Q48_FutureImpact_Secure",
        "params": {"map": future_secure_map, "unmapped_to_nan": True},
    },
    "47. Как ИИ повлияет на вашу защищенность и стабильность?": {
        "action": "map",
        "new_name": "Q48_FutureImpact_Secure",
        "params": {"map": future_secure_map, "unmapped_to_nan": True},
    },
    "49. Как ИИ изменит процесс обучения и творческой деятельности?": {
        "action": "map",
        "new_name": "Q49_FutureImpact_Learn",
        "params": {"map": future_learn_map, "unmapped_to_nan": True},
    },
    "48. Как ИИ изменит процесс обучения и творческой деятельности?": {
        "action": "map",
        "new_name": "Q49_FutureImpact_Learn",
        "params": {"map": future_learn_map, "unmapped_to_nan": True},
    },
    # --- Блок 3 Продолжение: Q50 (Мотивация) и Q51 (Условия обучения) - обработка КАЖДОГО пункта отдельно ---
    # Q50: Мотивация (Numeric)
    "49. Представьте, что на работе вам необходимо взаимодействовать с ИИ. Оцените, насколько указанные факторы могут усилить вашу мотивацию использовать ИИ. Возможность внести вклад в развитие передовых технологий и инноваций.": {
        "action": "numeric",
        "new_name": "Q50_Motiv_contribution",
    },
    "49. Представьте, что на работе вам необходимо взаимодействовать с ИИ. Оцените, насколько указанные факторы могут усилить вашу мотивацию использовать ИИ. Возможность передать рутинные задачи ИИ для занятия более творческими задачами.": {
        "action": "numeric",
        "new_name": "Q50_Motiv_routine_creative",
    },
    "49. Представьте, что на работе вам необходимо взаимодействовать с ИИ. Оцените, насколько указанные факторы могут усилить вашу мотивацию использовать ИИ. Возможность работать более эффективно и достигать лучших результатов.": {
        "action": "numeric",
        "new_name": "Q50_Motiv_efficiency",
    },
    "49. Представьте, что на работе вам необходимо взаимодействовать с ИИ. Оцените, насколько указанные факторы могут усилить вашу мотивацию использовать ИИ. Обучение новому и профессиональное развитие в области ИИ.": {
        "action": "numeric",
        "new_name": "Q50_Motiv_learning",
    },
    "49. Представьте, что на работе вам необходимо взаимодействовать с ИИ. Оцените, насколько указанные факторы могут усилить вашу мотивацию использовать ИИ. Повысить контроль над рабочими процессами и результатами.": {
        "action": "numeric",
        "new_name": "Q50_Motiv_control",
    },
    "49. Представьте, что на работе вам необходимо взаимодействовать с ИИ. Оцените, насколько указанные факторы могут усилить вашу мотивацию использовать ИИ. Повышение своей профессиональной ценности и востребованности на рынке труда.": {
        "action": "numeric",
        "new_name": "Q50_Motiv_prof_value",
    },
    "49. Представьте, что на работе вам необходимо взаимодействовать с ИИ. Оцените, насколько указанные факторы могут усилить вашу мотивацию использовать ИИ. Снижение рабочей нагрузки, поскольку часть задач переданы ИИ.": {
        "action": "numeric",
        "new_name": "Q50_Motiv_workload_reduction",
    },
    # Q51: Условия Обучения (MC)
    "50. Представьте, что на работе вам нужно обучаться взаимодействию с ИИ. Какие условия сделали бы обучение для вас наиболее привлекательным? Возможность использовать полученные навыки в текущих рабочих задачах для увеличения результатов.": {
        "action": "mc_group",
        "base_name": "Q51_LearnCond_apply_work",
    },  # base_name формирует имя колонки как Q51_LearnCond_apply_work
    "50. Представьте, что на работе вам нужно обучаться взаимодействию с ИИ. Какие условия сделали бы обучение для вас наиболее привлекательным? Возможность применять полученные знания на практике вне работы.": {
        "action": "mc_group",
        "base_name": "Q51_LearnCond_apply_outside",
    },
    "50. Представьте, что на работе вам нужно обучаться взаимодействию с ИИ. Какие условия сделали бы обучение для вас наиболее привлекательным? Возможность учиться в удобное для меня время и в удобном темпе.": {
        "action": "mc_group",
        "base_name": "Q51_LearnCond_flexible_time",
    },
    "50. Представьте, что на работе вам нужно обучаться взаимодействию с ИИ. Какие условия сделали бы обучение для вас наиболее привлекательным? Поддержка и помощь со стороны коллег и более опытных пользователей ИИ.": {
        "action": "mc_group",
        "base_name": "Q51_LearnCond_peer_support",
    },
    "50. Представьте, что на работе вам нужно обучаться взаимодействию с ИИ. Какие условия сделали бы обучение для вас наиболее привлекательным? Признание и поощрение со стороны руководства за освоение новых навыков.": {
        "action": "mc_group",
        "base_name": "Q51_LearnCond_recognition",
    },
    # Q51: Шкала Лжи (Drop) - Используем точный ключ
    "51. Как часто вы анализируете информацию, используя дополнительные источники для подтверждения ее достоверности?": {
        "action": "drop"
    },  # Сознательно дропаем шкалу лжи на данном этапе
    # ==========================================================================
    # --- Блоки 4-7: Доверие, Тревога, Идеал, Этика (Q53-Q71) ---
    # (Вопросы ТОЛЬКО из Анкеты 2, используем точные ключи)
    # ==========================================================================
    # Q53: Готовность делиться информацией (Grid)
    "26. Представьте, что ИИ может предложить вам персонализированную помощь и поддержку. Оцените, насколько вы будете готовы поделиться с ИИ следующей информацией.": {
        "action": "numeric_grid",
        "new_name": "Q53_Share",
    },
    # Q54: Доверие ИИ как проф. инструменту (Map)
    "27. Оцените, насколько на данный момент вы доверяете ИИ как точному и надежному инструменту в вашей профессиональной области.": {
        "action": "map",
        "new_name": "Q54_TrustAIProfTool",
        "params": {"map": trust_map_l2, "unmapped_to_nan": True},
    },
    # Q55: Изменение доверия (Map Grid)
    "28. Как изменится ваше доверие к ИИ в следующих случаях?": {
        "action": "map_grid",
        "new_name": "Q55_TrustChange",
        "params": {"map": trust_change_map_l2, "unmapped_to_nan": True},
    },
    # Q57: Тревожность (Grid)
    "30. Оцените, насколько вас тревожат следующие потенциальные проблемы в работе ИИ.": {
        "action": "numeric_grid",
        "new_name": "Q57_Anxiety",
    },
    # Q58: Опасения о будущем (Map)
    "31. Есть ли у вас опасения, связанные с развитием ИИ в настоящее время и в ближайшем будущем?": {
        "action": "map",
        "new_name": "Q58_AIFutureConcerns",
        "params": {"map": yes_no_map_l2, "unmapped_to_nan": True},
    },
    # Q59: Ранжирование тревог (Drop)
    "32. Расположите следующие утверждения от наименее тревожного (1) до наиболее тревожного (5) для вас.": {
        "action": "drop"
    },
    # Q60: Источник этических рисков (Map)
    "33. На ваш взгляд, с чем связаны этические риски внедрения технологий ИИ?": {
        "action": "map",
        "new_name": "Q60_EthicalRiskSource",
        "params": {"map": ethical_risk_source_map_l2, "unmapped_to_nan": True},
    },
    # Q61: Условия комфорта (MC)
    "34. Что сделает использование ИИ более комфортным для вас?": {
        "action": "mc_group",
        "base_name": "Q61_Comfort",
        "ignore_starts_with": ["Ничего из перечисленного"],
    },
    # Q63: Идеальное взаимодействие (Grid)
    "36. Насколько для вас важны следующие аспекты идеального взаимодействия с ИИ?": {
        "action": "numeric_grid",
        "new_name": "Q63_Ideal",
    },
    # Q64: Предпочтительный формат общения (Map)
    "37. Выберите наиболее комфортный для вас формат общения с ИИ.": {
        "action": "map",
        "new_name": "Q64_PrefCommFormat",
        "params": {"map": preferred_ai_comm_map_l2, "unmapped_to_nan": True},
    },
    # Q66: Нужны ли законы (Map)
    "39. Должны ли существовать законы, регулирующие работу ИИ?": {
        "action": "map",
        "new_name": "Q66_AIRegLawsNeeded",
        "params": {"map": yes_no_map_l2, "unmapped_to_nan": True},
    },
    # Q67: Моральные решения ИИ (Map)
    "40. Допустимо ли, чтобы ИИ принимал моральные решения (например, в ситуации, где автономный автомобиль должен выбрать, кого спасти – пассажира или пешехода)?": {
        "action": "map",
        "new_name": "Q67_AIMoralDecisionAllow",
        "params": {"map": yes_no_map_l2_moral, "unmapped_to_nan": True},
    },
    # Q68: Основа решений ИИ (Map)
    "41. При принятии решений, чем должен руководствоваться ИИ?": {
        "action": "map",
        "new_name": "Q68_AIDecisionBasis",
        "params": {"map": ai_decision_basis_map_l2, "unmapped_to_nan": True},
    },
    # Q70: Ответственность (MC)
    "42. Кто должен нести ответственность за действия ИИ?": {
        "action": "mc_group",
        "base_name": "Q70_Responsibility",
        "ignore_starts_with": ["Все из вышеперечисленных"],
    },
    # Q71: Доверие институтам (MC)
    "43. Каким социальным институтам вы бы доверили разработку и регулирование ИИ?": {
        "action": "mc_group",
        "base_name": "Q71_TrustInst",
        "ignore_starts_with": ["Никакие из перечисленных"],
    },
}

standard_drop_patterns = []
text_block_identifiers = []

# --- 6. Запуск Обработки ---
# 1. Загрузка двух листов
df1 = load_sheet(file_name, sheet1_name)
df2 = load_sheet(file_name, sheet2_name)

if df1 is not None and df2 is not None:
    logging.info("Объединение листов через CONCAT...")
    # Просто объединяем, ничего не удаляем
    df_combined = pd.concat([df1, df2], ignore_index=True, sort=False)

    # 2. Назначаем participant_id (уникальный для всей совокупности)
    df_combined = df_combined.reset_index(drop=True)
    df_combined["participant_id"] = np.arange(len(df_combined))

    # 3. Удаляем старый id (не нужен, есть participant_id)
    if id_col in df_combined.columns:
        df_combined = df_combined.drop(columns=[id_col])

    # 4. Переносим participant_id в начало
    cols = ["participant_id"] + [
        c for c in df_combined.columns if c != "participant_id"
    ]
    df_combined = df_combined[cols]

    print(
        f"Назначено новых participant_id: {df_combined['participant_id'].min()} - {df_combined['participant_id'].max()}"
    )
    print(f"Финальный размер после очистки и concat: {df_combined.shape}")

    # ===================
    # --- PIPELINE ---
    # ===================

    # a. Первичная очистка (убирает мусорные и пустые столбцы, служебные и т.д.)
    df_cleaned = initial_cleanup(
        df_combined, standard_drop_patterns, text_block_identifiers
    )

    # b. Применяем карту обработки (перевод значений, создание новых столбцов и пр.)
    df_processed, processed_cols = apply_processing_map_v37_concat(
        df_cleaned, processing_map
    )
    df_processed = df_processed.dropna(axis=1, how="all")

    # c. Динамическая сортировка колонок для удобства анализа
    df_processed_sorted = sort_columns_dynamically(df_processed)

    # d. Генерация "портретов" переменных — анализ зависимостей
    # generate_variable_portraits(df_processed_sorted, "Данные: итоговая версия")


else:
    logging.error("Не удалось загрузить один или оба листа.")

    # Дальше уже работаешь с df_combined как с чистым источником!
    # ... pipeline: initial_cleanup, apply_processing_map, sort_columns, анализ и т.д. ...

# %%
# ==============================================================================

Основные импорты (прямые) и настройки выполнены.
Назначено новых participant_id: 0 - 1921
Финальный размер после очистки и concat: (1922, 260)


In [ ]:
# ==============================================================================
# --- ЭТАП R1: Импутация, портреты, PCA-установки, частота ИИ, метафора ---
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

# -------------------------------------------
# 1. Подготовка: выборка, кодирование, KNN
# -------------------------------------------
df_final_portraits = df_processed_sorted.copy()  # Исходная таблица

# --- Определяем, какие колонки нужны для импутации ---
cols_pca_base = [
    c
    for c in df_final_portraits.columns
    if any(
        [
            c.startswith("Q13_GM_"),
            c.startswith("Q14_RC_"),
            c.startswith("Q23_UsagePurpose_"),
            c.startswith("Q29_Eval_"),
            c.startswith("Q33_UsagePurpose_"),
            c.startswith("Q34_NotDiscuss_"),
            c.startswith("Q35_Emotion_"),
            c.startswith("Q44_AICanCritique"),
            c.startswith("Q45_Q49_FutureImpact_"),
            c.startswith("Q50_Motiv_"),
            c.startswith("Q61_Comfort_"),
        ]
    )
]

cols_metaphor = [c for c in df_final_portraits.columns if c.startswith("Q32_Metaphor_")]
cols_impute = cols_pca_base + cols_metaphor

# Категориальные для кодирования в KNN
cat_cols_for_knn = [
    "Q2_AgeGroup_processed",
    "Q3_Gender_processed",
    "Q4_LocationType_processed",
    "Q5_MaritalStatus_processed",
    "Q6_ChildrenStatus_processed",
    "Q7_EducationLevel_processed",
    "Q9_Position_processed",
    "Q11_IncomeLevel_processed",
    "Q12_WellbeingLevel_processed",
]
cat_cols_for_knn = [c for c in cat_cols_for_knn if c in df_final_portraits.columns]

# Подготавливаем датасет для KNN
df_knn = df_final_portraits[cols_impute + cat_cols_for_knn].copy()
ordinal_encoders = {}

# Кодируем категориальные
for col in cat_cols_for_knn:
    enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)
    not_nan = df_knn[col].notna()
    if not_nan.sum() > 0:
        enc.fit(df_knn.loc[not_nan, [col]])
        df_knn.loc[not_nan, col] = enc.transform(df_knn.loc[not_nan, [col]]).flatten()
        ordinal_encoders[col] = enc

# Стандартизируем
scaler = StandardScaler()
df_knn_scaled = pd.DataFrame(
    scaler.fit_transform(df_knn), columns=df_knn.columns, index=df_knn.index
)

# --- Импутируем KNN ---
imputer = KNNImputer(n_neighbors=5)
df_knn_imputed_scaled = pd.DataFrame(
    imputer.fit_transform(df_knn_scaled), columns=df_knn.columns, index=df_knn.index
)

# Обратно к реальным значениям
df_knn_imputed = pd.DataFrame(
    scaler.inverse_transform(df_knn_imputed_scaled),
    columns=df_knn.columns,
    index=df_knn.index,
)

# Возвращаем в портреты только имутированные PCA/метафора колонки
for col in cols_impute:
    if col in df_knn_imputed:
        df_final_portraits[col] = df_knn_imputed[col]

# -------------------------------------------
# 2. PCA для установок (с новыми лейблами)
# -------------------------------------------
# Массив для PCA
pca_data = df_final_portraits[cols_pca_base].fillna(0).to_numpy()

pca = PCA(n_components=4)
components = pca.fit_transform(pca_data)
explained_var = pca.explained_variance_ratio_
cumulative_var = explained_var.cumsum()

# Сохраняем компоненты в датафрейм
for i in range(4):
    df_final_portraits[f"Установка_PCA_{i+1}"] = components[:, i]

# Лейблы категорий (можно поправить под свой смысл!)
cat_labels = [
    "Стремление к развитию и интересу (Позитивные установки)",
    "Негативные эмоции, избегают обсуждений с ИИ",
    "Фокус на контроле, прозрачности и безопасности",
    "Испытывают трудности, сопротивляются новому",
]
cat_order = np.array([0, 2, 1, 3])  # соответствие меток порядку
df_final_portraits["Установка_Категория"] = pd.Categorical(
    [cat_labels[i] for i in components.argmax(axis=1)],
    categories=cat_labels,
    ordered=True,
)

# -------------------------------------------
# 3. "Частота взаимодействия с ИИ"
# -------------------------------------------
# Сохраняем как отдельную категорию
if "Q22_AITechUseFreq" in df_final_portraits.columns:
    freq_labels = {1: "Никогда", 2: "Редко", 3: "Иногда", 4: "Часто", 5: "Очень часто"}
    df_final_portraits["Частота взаимодействия с ИИ"] = df_final_portraits[
        "Q22_AITechUseFreq"
    ].map(freq_labels)
    df_final_portraits["Частота взаимодействия с ИИ"] = df_final_portraits[
        "Частота взаимодействия с ИИ"
    ].fillna("Нет данных")

# -------------------------------------------
# 4. Импутация метафоры восприятия ИИ
# -------------------------------------------
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder

if any(col.startswith("Q32_Metaphor_") for col in df_final_portraits.columns):
    metaphor_cols = [
        col for col in df_final_portraits.columns if col.startswith("Q32_Metaphor_")
    ]
    metaphor_rounded = df_final_portraits[metaphor_cols].round()
    most_likely_metaphor_col_names = metaphor_rounded.idxmax(axis=1)
    max_values = metaphor_rounded.max(axis=1)
    most_likely_metaphor_col_names[(max_values <= 0) | max_values.isna()] = np.nan
    prefix_to_remove = "Q32_Metaphor_чем_для_вас_является_ии_"
    sign_value_raw = most_likely_metaphor_col_names.dropna().str.replace(
        prefix_to_remove, "", regex=False
    )
    metaphor_short_labels = {
        "велосипед_для_разума": "Велосипед для разума",
        "демон_в_процессоре": "Демон в процессоре",
        "джинн_из_бутылки": "Джинн из бутылки",
        "инструмент": "Инструмент",
        "конкурент": "Конкурент",
        "партнер": "Партнер",
        "черный_ящик": "Черный ящик",
    }
    sign_value_mapped = sign_value_raw.map(metaphor_short_labels)
    sign_value_mapped = sign_value_mapped.fillna(sign_value_raw)
    df_final_portraits.loc[sign_value_mapped.index, "SignValue_Imputed"] = (
        sign_value_mapped
    )


cat_features = [
    col
    for col in [
        "Установка_Категория",
        "Частота взаимодействия с ИИ",
        "Q2_AgeGroup_processed",
        "Q3_Gender_processed",
        "Q4_LocationType_processed",
        "Q5_MaritalStatus_processed",
    ]
    if col in df_final_portraits.columns
]


df_train = df_final_portraits[
    df_final_portraits["SignValue_Imputed"].notna()
    & df_final_portraits["Установка_Категория"].notna()
]
df_predict = df_final_portraits[
    df_final_portraits["SignValue_Imputed"].isna()
    & df_final_portraits["Установка_Категория"].notna()
]

if not df_train.empty and not df_predict.empty:
    enc = OrdinalEncoder()
    X_train = enc.fit_transform(df_train[cat_features])
    y_train = df_train["SignValue_Imputed"].astype(str).values

    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)

    X_pred = enc.transform(df_predict[cat_features])
    y_pred = knn.predict(X_pred)

    df_final_portraits.loc[df_predict.index, "SignValue_Imputed"] = y_pred

# Мода на крайний случай
df_final_portraits["SignValue_Imputed"] = df_final_portraits[
    "SignValue_Imputed"
].fillna(df_final_portraits["SignValue_Imputed"].mode().iloc[0])


# -------------------------------------------
# 5. Вывод информации
# -------------------------------------------
print("=" * 40)
print(f"PCA explained variance ratio: {np.round(explained_var, 4)}")
print(f"PCA cumulative variance: {np.round(cumulative_var, 4)}")
print("\n--- Распределение по категориям установок (max PCA): ---")
print(df_final_portraits["Установка_Категория"].value_counts())

print("\n--- Статистика по компонентам (mean, std, min, max): ---")
print(df_final_portraits[[f"Установка_PCA_{i+1}" for i in range(4)]].describe().T)

for i in range(4):
    comp_weights = pd.Series(
        pca.components_[i], index=cols_pca_base, name=f"PCA_{i+1}"
    ).sort_values(ascending=False)
    print(f"\nТоп-5 переменных для компоненты {i+1} (+)")
    print(comp_weights.head(5))

print("\n--- Корреляция компонент между собой ---")
print(df_final_portraits[[f"Установка_PCA_{i+1}" for i in range(4)]].corr())

print("\n--- Распределение по частоте взаимодействия с ИИ: ---")
if "Частота взаимодействия с ИИ" in df_final_portraits.columns:
    print(df_final_portraits["Частота взаимодействия с ИИ"].value_counts())
else:
    print("Колонка с частотой взаимодействия не найдена!")

print("\n--- Распределение по метафорам (SignValue_Imputed): ---")
if "SignValue_Imputed" in df_final_portraits.columns:
    print(df_final_portraits["SignValue_Imputed"].value_counts(dropna=False))
else:
    print("SignValue_Imputed не найден!")

# --- Проверка пропусков
for col in [f"Установка_PCA_{i+1}" for i in range(4)] + [
    "Установка_Категория",
    "Частота взаимодействия с ИИ",
    "SignValue_Imputed",
]:
    if col in df_final_portraits.columns:
        print(f"{col}: пропусков = {df_final_portraits[col].isna().sum()}")

print("\nГотово! df_final_portraits готов.")
# ==============================================================================

PCA explained variance ratio: [0.2633 0.1743 0.0785 0.0522]
PCA cumulative variance: [0.2633 0.4377 0.5162 0.5684]

--- Распределение по категориям установок (max PCA): ---
Установка_Категория
Стремление к развитию и интересу (Позитивные установки)    692
Негативные эмоции, избегают обсуждений с ИИ                536
Испытывают трудности, сопротивляются новому                363
Фокус на контроле, прозрачности и безопасности             331
Name: count, dtype: int64

--- Статистика по компонентам (mean, std, min, max): ---
                  count          mean       std       min       25%       50%  \
Установка_PCA_1  1922.0  7.098034e-16  2.691202 -6.325140 -1.922153  0.269206   
Установка_PCA_2  1922.0  3.249568e-15  2.189695 -9.636769 -1.334587 -0.325136   
Установка_PCA_3  1922.0 -1.449182e-15  1.469235 -6.462356 -0.975621 -0.051727   
Установка_PCA_4  1922.0 -5.323525e-16  1.198381 -3.409419 -0.878385 -0.017391   

                      75%        max  
Установка_PCA_1  1.896998 

In [ ]:
import numpy as np
import pandas as pd
import pingouin as pg
from factor_analyzer import FactorAnalyzer
from IPython.display import display
from sklearn.impute import SimpleImputer
import re

def add_scale_means(df, likert_bases):
    for base in likert_bases:
        cols = [c for c in df.columns if c.startswith(base + "_")]
        if cols:
            df[f"{base}_Mean"] = df[cols].mean(axis=1, skipna=True)
    return df

def run_efa(df, efa_cols, n_factors=5, rotation="promax", alpha_thresh=0.35):
    data = SimpleImputer(strategy="median").fit_transform(df[efa_cols])
    fa = FactorAnalyzer(n_factors=n_factors, rotation=rotation, method="ml")
    fa.fit(data)
    loadings = pd.DataFrame(fa.loadings_, index=efa_cols)
    print("Объяснённая дисперсия:")
    print(fa.get_factor_variance())
    for i, factor in enumerate(loadings.columns):
        vars_in_factor = loadings.index[loadings[factor].abs() > alpha_thresh]
        if len(vars_in_factor) >= 2:
            alpha, ci = pg.cronbach_alpha(
                data=pd.DataFrame(data, columns=efa_cols)[vars_in_factor]
            )
            print(f"F{i+1}: альфа={alpha:.3f}, 95% CI {ci}")
        else:
            print(f"F{i+1}: слишком мало переменных для альфы.")
    factor_scores = fa.transform(data)
    score_cols = [f"F{i+1}" for i in range(factor_scores.shape[1])]
    df_scores = pd.DataFrame(factor_scores, columns=score_cols, index=df.index)
    return fa, loadings, df_scores

# ВСЕ числовые переменные с Q13+ (для факторного анализа)
efa_cols = [
    c for c in df_final_portraits.columns
    if re.match(r'^Q(1[3-9]|[2-9][0-9]|[1-9][0-9]{2,})_', c)
    and pd.api.types.is_numeric_dtype(df_final_portraits[c])
    and df_final_portraits[c].nunique(dropna=True) > 1
]

print(f"Количество переменных для EFA: {len(efa_cols)}")

# 1. Оставить только столбцы с нормальной дисперсией
from sklearn.feature_selection import VarianceThreshold
thresh = 0.05
selector = VarianceThreshold(threshold=thresh)
selected = selector.fit_transform(df_final_portraits[efa_cols])
efa_cols_good = [c for c, v in zip(efa_cols, selector.get_support()) if v]
print(f"Отобрано переменных с нормальной дисперсией: {len(efa_cols_good)}")

n_factors = 9  # или больше, если данных много
fa, loadings, df_scores = run_efa(df_final_portraits, efa_cols_good, n_factors=n_factors, rotation="varimax")

for col in df_scores.columns:
    df_final_portraits[col] = df_scores[col]

print("Факторные оценки успешно сохранены в df_final_portraits")

threshold = 0.35
loadings_filtered = loadings.where(loadings.abs() > threshold, "")  # все, что по модулю ≤ threshold, заменено на ""

# ВОТ ЭТО отображай:
display(loadings_filtered.style.background_gradient(cmap="RdBu", axis=None, vmin=-1, vmax=1))

Количество переменных для EFA: 219
Отобрано переменных с нормальной дисперсией: 197


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Объяснённая дисперсия:
(array([25.05224637, 12.76466682,  8.26092713,  6.80395532,  5.5932362 ,
        5.1737746 ,  4.48519843,  3.93256775,  3.70627706]), array([0.12716876, 0.06479526, 0.04193364, 0.03453784, 0.02839206,
       0.02626282, 0.0227675 , 0.01996227, 0.01881359]), array([0.12716876, 0.19196403, 0.23389767, 0.26843551, 0.29682757,
       0.32309039, 0.34585789, 0.36582017, 0.38463375]))
F1: альфа=0.821, 95% CI [0.81  0.833]
F2: альфа=0.947, 95% CI [0.943 0.95 ]
F3: альфа=0.864, 95% CI [0.855 0.872]
F4: альфа=0.917, 95% CI [0.912 0.923]
F5: альфа=0.886, 95% CI [0.879 0.894]
F6: альфа=0.863, 95% CI [0.853 0.872]
F7: альфа=0.889, 95% CI [0.882 0.897]
F8: альфа=0.935, 95% CI [0.93  0.939]
F9: альфа=0.870, 95% CI [0.861 0.879]
Факторные оценки успешно сохранены в df_final_portraits


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
<ipython-input-6-8d563bc664a1>:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final_portraits[col] = df_scores[col]
<ipython-input-6-8d563bc664a1>:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final_portraits[col] = df_scores[col]


,0,1,2,3,4,5,6,7,8
Q13_GM_критика_моих_навыков_работы_с_новыми_технологиями_помогает_мне_ст,,,,,,0.639774,,,
Q13_GM_неудачи_при_освоении_технологий_не_пугают_меня_а_наоборот_стимули,,,,,,0.649448,,,
Q13_GM_я_верю_что_могу_стать_экспертом_в_области_использования_технологи,,,,,,0.659511,,,
Q13_GM_я_люблю_пробовать_новые_подходы_и_технологии_в_своей_работе_даже_,,,,,,0.692705,,,
Q13_GM_я_стремлюсь_развивать_свои_способности_к_освоению_новых_технологи,,,,,,0.688570,,,
Q14_RC_изменения_связанные_с_внедрением_нового_вызывают_у_меня_скорее_бе,,,,,0.659386,,,,
Q14_RC_я_опасаюсь_что_внедрение_новых_технологий_нарушит_привычный_и_ком,,,,,0.662114,,,,
Q14_RC_я_предпочитаю_работать_с_проверенными_и_знакомыми_методами_чем_ос,,,,,0.533777,,,,
Q14_RC_я_скорее_буду_сопротивляться_изменениям_чем_активно_участвовать_в,,,,,0.676164,,,,
Q14_RC_я_считаю_что_текущие_методы_моей_работы_вполне_эффективны_и_не_тр,,,,,0.486931,,,,


## Описание факторов (Varimax, 9 факторов)

### **F1 — Практическое использование и влияние ИИ**
- Включает: опыт применения ИИ, влияние на разные сферы жизни, использование ИИ для работы, учёбы, быта, развлечений, общения и саморазвития.
- Примеры переменных:  
  - Q23_EncounterArea_развлечения_и_медиа  
  - Q33_UsagePurpose_для_развлечения  
  - Q25_Influence_мой_личный_опыт_использования_ии  
  - Q43_HumanSup_общение_с_людьми_эмоциональная_поддержка  
- **Смысл:** отражает широту и интенсивность интеграции ИИ в повседневную и профессиональную деятельность.

---

### **F2 — Доверие, антропоморфизм и социальное принятие ИИ**
- Включает: доверие к ИИ в различных задачах, ожидание человеческих черт, готовность делегировать ИИ контроль, креативные и этические задачи.
- Примеры переменных:  
  - Q41_Trust_оцените_насколько_вы_бы_доверили_ему_следующие_задачи  
  - Q38_Anth_ии_должен_иметь_человеческие_черты  
  - Q38_Anth_ии_это_личность_с_характером_и_стилем_общения  
- **Смысл:** отражает уровень доверия, ожидания антропоморфизма и социальную интеграцию ИИ.

---

### **F3 — Знание и опыт взаимодействия с ИИ**
- Включает: знание технологий, частота использования, понимание принципов работы, опыт взаимодействия с различными ИИ-системами.
- Примеры переменных:  
  - Q19_ConsideredAI_интеллектуальные_поисковые_системы  
  - Q21_Interact_интеллектуальные_поисковые_системы  
  - Q18_AIUnderstanding  
- **Смысл:** отражает осведомлённость, информированность и реальный опыт взаимодействия с ИИ.

---

### **F4 — Готовность делиться личной информацией и открытость**
- Включает: готовность делиться разными типами информации с ИИ, доверие к ИИ-инструментам, открытость к новым форматам коммуникации.
- Примеры переменных:  
  - Q53_Share_информация_о_ваших_проблемах  
  - Q53_Share_информация_о_ваших_отношениях_с_другими_людьми  
  - Q54_TrustAIProfTool  
- **Смысл:** отражает степень открытости и доверия к ИИ в вопросах обмена личной информацией.

---

### **F5 — Сопротивление изменениям и тревожность по поводу ИИ**
- Включает: опасения и тревоги, связанные с внедрением новых технологий, предпочтение привычных методов, негативные эмоции при работе с ИИ.
- Примеры переменных:  
  - Q14_RC_я_опасаюсь_что_внедрение_новых_технологий_нарушит_привычный_и_комфортный_уклад  
  - Q29_Eval_мне_некомфортно_работать_с_ии  
  - Q14_RC_я_предпочитаю_работать_с_проверенными_и_знакомыми_методами  
- **Смысл:** отражает уровень тревожности, консерватизм и сопротивление технологическим изменениям.

---

### **F6 — Готовность к обучению и освоению новых технологий**
- Включает: установки на развитие, стремление к освоению новых технологий, вера в свои способности, интерес к инновациям.
- Примеры переменных:  
  - Q13_GM_я_стремлюсь_развивать_свои_способности_к_освоению_новых_технологий  
  - Q13_GM_я_люблю_пробовать_новые_подходы  
  - Q13_GM_я_верю_что_могу_стать_экспертом  
- **Смысл:** отражает проактивную позицию в отношении саморазвития и освоения ИИ.

---

### **F7 — Ожидания эффективности, контроля и безопасности**
- Включает: ожидания высокого уровня защиты данных, прозрачности, контроля, эффективности и инновационности ИИ.
- Примеры переменных:  
  - Q63_Ideal_конфиденциальность_и_безопасность  
  - Q63_Ideal_контроль_и_управляемость  
  - Q63_Ideal_прозрачность_работы_ии  
- **Смысл:** отражает требования к качеству, контролю и безопасности при использовании ИИ.

---

### **F8 — Мотивация и выгоды от использования ИИ**
- Включает: ожидания профессиональной пользы, повышения эффективности, снижения рутины, профессионального развития.
- Примеры переменных:  
  - Q50_Motiv_control  
  - Q50_Motiv_efficiency  
  - Q50_Motiv_prof_value  
- **Смысл:** отражает внутреннюю мотивацию и прагматические выгоды от интеграции ИИ.

---

### **F9 — Тревожность и опасения по поводу рисков ИИ**
- Включает: тревога из-за непрозрачности, утраты контроля, этических и правовых рисков, опасения по поводу последствий ИИ.
- Примеры переменных:  
  - Q57_Anxiety_непредсказуемость_решений  
  - Q57_Anxiety_ограничение_контроля  
  - Q57_Anxiety_этические_и_правовые_риски_ии  
- **Смысл:** отражает выраженную тревожность и опасения, связанные с рисками и неопределённостью в применении ИИ.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
import plotly.graph_objects as go

MIN_CLUSTER_SIZE = 100
MAX_CLUSTER_SIZE = 250
INIT_CLUSTERS = len(df_final_portraits) // ((MIN_CLUSTER_SIZE + MAX_CLUSTER_SIZE) // 2)
factor_cols = [c for c in df_final_portraits.columns if c.startswith("F") or "factor" in c.lower()]
attitude_col = "Установка_Категория"
metaphor_col = "SignValue_Imputed"

df_cluster = df_final_portraits.dropna(subset=factor_cols).copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster[factor_cols])

# Первичная кластеризация
kmeans = KMeans(n_clusters=INIT_CLUSTERS, n_init=100, random_state=42)
df_cluster["Cluster"] = kmeans.fit_predict(X_scaled)

# --- Слияние малых, дробление крупных кластеров ---
def adjust_clusters(df, min_size, max_size, factor_cols):
    next_label = df['Cluster'].max() + 1
    # 1. Сливаем маленькие
    while True:
        cluster_centers = df.groupby('Cluster')[factor_cols].mean()
        cluster_sizes = df['Cluster'].value_counts()
        small_clusters = cluster_sizes[cluster_sizes < min_size].index.tolist()
        if not small_clusters:
            break
        for small in small_clusters:
            if len(cluster_centers) <= 1:
                continue
            dists = cdist([cluster_centers.loc[small]], cluster_centers.drop(small))
            nearest = cluster_centers.drop(small).index[dists.argmin()]
            df.loc[df['Cluster'] == small, 'Cluster'] = nearest

    # 2. Дробим большие
    while True:
        cluster_sizes = df['Cluster'].value_counts()
        large_clusters = cluster_sizes[cluster_sizes > max_size].index.tolist()
        if not large_clusters:
            break
        for large in large_clusters:
            subset = df[df['Cluster'] == large]
            n_sub = int(np.ceil(len(subset) / max_size))
            if n_sub <= 1:
                continue
            km = KMeans(n_clusters=n_sub, n_init=20, random_state=42)
            sub_labels = km.fit_predict(subset[factor_cols])
            # Новые кластеры получат уникальные лейблы
            for i in range(n_sub):
                new_label = next_label
                next_label += 1
                idx = subset.index[sub_labels == i]
                df.loc[idx, 'Cluster'] = new_label
    return df

df_cluster = adjust_clusters(df_cluster, MIN_CLUSTER_SIZE, MAX_CLUSTER_SIZE, factor_cols)

# Перенумеровка кластеров по порядку
unique_clusters = {old: i for i, old in enumerate(sorted(df_cluster['Cluster'].unique()))}
df_cluster['Cluster'] = df_cluster['Cluster'].map(unique_clusters)

# Присоединение к исходному датафрейму
df_final_portraits = df_final_portraits.merge(
    df_cluster[["Cluster"]], left_index=True, right_index=True, how="left"
)
df_final_portraits["Cluster"] = df_cluster["Cluster"]

# --- Группировка и профилирование ---
grouped = df_final_portraits.groupby("Cluster")
factor_means = grouped[factor_cols].mean().round(2)
attitude_counts = grouped[attitude_col].agg(lambda x: x.value_counts().head(3).to_dict())
metaphor_counts = grouped[metaphor_col].agg(lambda x: x.value_counts().head(3).to_dict())
counts = grouped.size().rename("N")
cluster_description = pd.concat([
    counts, factor_means,
    attitude_counts.rename("Топ установки"),
    metaphor_counts.rename("Топ метафоры"),
], axis=1).sort_values("N", ascending=False)

# --- Визуализация PCA ---
pca = PCA(n_components=2)
coords = pca.fit_transform(df_cluster[factor_cols])
df_cluster['PCA1'], df_cluster['PCA2'] = coords[:, 0], coords[:, 1]

# Цвета и символы
symbols = ["circle", "square", "diamond", "cross", "triangle-up", "triangle-down", "star", "hexagram", "hourglass", "pentagon", "triangle-left", "triangle-right"] * 10
palette = ["#636EFA", "#EF553B", "#00CC96", "#AB63FA", "#FFA15A", "#19D3F3", "#FF6692", "#B6E880", "#FF97FF", "#FECB52", "#C2C2C2", "#9D9D9D"] * 10
cluster_ids = sorted(df_cluster['Cluster'].unique())
symbol_map = {cid: symbols[i] for i, cid in enumerate(cluster_ids)}
color_map = {cid: palette[i] for i, cid in enumerate(cluster_ids)}

traces = []
for cid in cluster_ids:
    d = df_cluster[df_cluster['Cluster'] == cid]
    traces.append(go.Scatter(
        x=d['PCA1'],
        y=d['PCA2'],
        mode='markers',
        marker_symbol=symbol_map[cid],
        marker=dict(size=13, color=color_map[cid], line=dict(width=1.5, color="black")),
        name=f'Кластер {cid} (N={len(d)})'
    ))

fig = go.Figure(traces)
fig.update_layout(
    title="Визуализация кластеров (100 ≤ N ≤ 250 в каждом)",
    template='plotly_white',
    width=950, height=670,
    legend=dict(font=dict(size=12)),
)
fig.show()

print(df_cluster['Cluster'].value_counts().sort_index())


Cluster
0     221
1     184
2     218
3     178
4     143
5     213
6     100
7     177
8     234
9      97
10    157
Name: count, dtype: int64


In [ ]:
# =========== Вспомогательные функции ===========
def _clean_string_for_mapping(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower().strip()
    text = text.replace("\xa0", " ")
    text = text.replace("–", "-").replace("—", "-")
    text = text.replace(".", " ")
    text = re.sub(r"\(.*?\)", "", text).strip()
    text = text.rstrip(".,:;?!")
    text = re.sub(r"(?<=\d)\.(?=\d)", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def create_cleaned_map(map_dict):
    if not isinstance(map_dict, dict):
        return map_dict
    return {
        _clean_string_for_mapping(k): v for k, v in map_dict.items() if not pd.isna(k)
    }


def get_category_counts_str(series):
    counts = series.value_counts(dropna=True)
    return ", ".join(f"{cat}: {cnt}" for cat, cnt in counts.items())


# =========== Справочники демографии ===========
age_map = create_cleaned_map(
    {
        "до 18 лет": 0.0,
        "18 - 24 года": 1.0,
        "25 - 34 года": 2.0,
        "25 - 34 лет": 2.0,
        "35 - 44 года": 3.0,
        "35 - 44 лет": 3.0,
        "45 - 64 года": 4.0,
        "45 - 64 лет": 4.0,
        "65 лет и старше": 5.0,
    }
)
gender_map = create_cleaned_map({"мужской": 0.0, "женский": 1.0})
location_map = create_cleaned_map(
    {
        "москва, санкт-петербург": 1.0,
        "город-миллионник, кроме москвы и санкт-петербурга": 2.0,
        "город с населением от 50 тыс до 1 миллиона человек": 3.0,
        "город с населением от 50 тыс до 1 млн чел": 3.0,
        "населенный пункт с населением до 50 тыс человек": 4.0,
    }
)
marital_map = create_cleaned_map(
    {
        "состою в браке / гражданском браке / отношениях": 1.0,
        "холост/разведен/вдовствую": 0.0,
    }
)
children_map = create_cleaned_map(
    {
        "нет": 0.0,
        "один ребенок": 1.0,
        "двое детей": 2.0,
        "трое и более детей": 3.0,
    }
)
income_map = create_cleaned_map(
    {
        "до 50 000 р": 1.0,
        "50 001 - 100 000 р": 2.0,
        "100 001 - 150 000 р": 3.0,
        "150 001 - 200 000 р": 4.0,
        "200 001 - 400 000 р": 5.0,
        "400 001 р и выше": 6.0,
    }
)
wellbeing_map = create_cleaned_map(
    {
        "денег не хватает даже на еду": 1.0,
        "на еду хватает, но покупка одежды вызывает затруднение": 2.0,
        "на одежду хватает, но покупка бытовой техники вызывает затруднение": 3.0,
        "на бытовую технику хватает, но покупка автомобиля вызывает затруднения": 4.0,
        "на автомобиль хватает": 5.0,
    }
)
education_map = create_cleaned_map(
    {
        "среднее общее и ниже": 1.0,
        "среднее специальное": 2.0,
        "высшее": 3.0,
    }
)
position_map_simple = create_cleaned_map(
    {
        "стажер": 1.0,
        "специалист": 2.0,
        "менеджер": 3.0,
        "руководитель отдела": 4.0,
        "директор / исполнительный директор": 5.0,
        "предприниматель / владелец бизнеса": 6.0,
        "преподаватель / научный сотрудник": 7.0,
        "студент": 8.0,
    }
)

ai_usage_freq_map = {
    "никогда": 1.0,
    "редко (раз в несколько месяцев)": 2.0,
    "иногда (несколько раз в месяц)": 3.0,
    "часто (несколько раз в неделю)": 4.0,
    "очень часто (ежедневно)": 5.0,
}


# =========== КОНСТАНТЫ ФАКТОРОВ ===========
factor_names = {
    "F1": "Практическое использование и влияние ИИ",
    "F2": "Доверие, антропоморфизм и социальное принятие ИИ",
    "F3": "Знание и опыт взаимодействия с ИИ",
    "F4": "Готовность делиться личной информацией и открытость",
    "F5": "Сопротивление изменениям и тревожность по поводу ИИ",
    "F6": "Готовность к обучению и освоению новых технологий",
    "F7": "Ожидания эффективности, контроля и безопасности",
    "F8": "Мотивация и выгоды от использования ИИ",
    "F9": "Тревожность и опасения по поводу рисков ИИ",
}
factor_cols = [c for c in df_final_portraits.columns if c.startswith("F")]

# =========== ДЕМОКОЛОНКИ ===========
demo_cols = [
    "Q2_AgeGroup",
    "Q3_Gender",
    "Q5_MaritalStatus",
    "Q6_ChildrenStatus",
    "Q7_EducationLevel",
    "Q9_Position",
    "Q11_IncomeLevel",
    "Q12_WellbeingLevel",
    "Q4_LocationType",
]

# =========== КЛАСТЕРЫ ===========
cluster_titles = {
    0: {
        "Название": "Осторожные теоретики",
        "Резюме": "Среднее доверие (F2=0.10) при низком использовании (F1=-1.20). Воспринимают ИИ как 'Инструмент' (121). Основная группа: 45-64 года (104 чел.), с высшим образованием (64%).",
    },
    1: {
        "Название": "Критичные скептики",
        "Резюме": "Крайне низкие ожидания безопасности (F7=-2.02). Активные мужчины (106 чел.) с низкой частотой использования. 52% имеют доход до 100 тыс.р.",
    },
    2: {
        "Название": "Опытные скептики",
        "Резюме": "Активное использование (F1=0.67) без доверия (F2=-0.95). 113 мужчин vs 105 женщин. 55% состоят в браке, 40% имеют детей.",
    },
    3: {
        "Название": "Активные прагматики",
        "Резюме": "Высокое использование (F1=1.21) и доверие (F2=1.04). 87 метафор 'Велосипед для разума'. 64% с доходом 50-150 тыс.р., 77% специалистов.",
    },
    4: {
        "Название": "Тревожные минималисты",
        "Резюме": "Низкое использование (F1=-0.83) с высокой тревожностью (F5=0.55). 75 женщин vs 68 мужчин. 85% с высшим образованием, 53% из мегаполисов.",
    },
    5: {
        "Название": "Активные сомневающиеся",
        "Резюме": "Частое использование (98 чел.) при низком понимании (F3=0.43). 107 женщин, 93 с детьми. 69% оценивают благосостояние как среднее.",
    },
    6: {
        "Название": "Малоактивные наблюдатели",
        "Резюме": "Самый большой кластер (356 чел.). Низкое использование (F1=0.87) при теоретическом интересе. 195 метафор 'Инструмент', 64% замужем/женаты.",
    },
    7: {
        "Название": "Осторожные коммуникаторы",
        "Резюме": "Готовы делиться информацией (F4=1.28) при редком использовании. 101 женщина, 75 из Москвы/СПб. 73% с доходом 50-100 тыс.р.",
    },
    8: {
        "Название": "Инновационные реалисты",
        "Резюме": "Баланс использования (F1=0.96) и осторожности (F9=-0.00). 234 чел., 120 женщин. 96 метафор 'Велосипед для разума', 44% руководителей.",
    },
    9: {
        "Название": "Вынужденные пользователи",
        "Резюме": "Минимальное использование (F1=-1.23) при профессиональной необходимости. 48% руководителей отделов, 56% из городов 50к-1млн. 73% с детьми.",
    },
    10: {
        "Название": "Сбалансированные прагматики",
        "Резюме": "Умеренное использование (F1=0.86) с фокусом на безопасности (94 упоминания). 157 чел., 89 женщин. 75 метафор 'Велосипед для разума', 52% из регионов.",
    }
}


In [ ]:
import re

import numpy as np
import pandas as pd

# =========== ГРУППИРОВКА ===========
grouped = df_final_portraits.groupby("Cluster")
profile_means = grouped[factor_cols].mean().rename(columns=factor_names).round(2)
freq_counts = grouped["Q22_AITechUseFreq"].agg(get_category_counts_str)
profile_counts = grouped.size().rename("N")

# --- Распределение по установкам и метафорам ---
dominant_counts = grouped["Установка_Категория"].agg(get_category_counts_str)
signvalue_counts = grouped["SignValue_Imputed"].agg(get_category_counts_str)

# --- Демография ---
demo_counts = {}
for col in demo_cols:
    if col in df_final_portraits.columns:
        demo_counts[col] = grouped[col].agg(get_category_counts_str)
    else:
        demo_counts[col] = pd.Series(
            ["Нет данных"] * grouped.ngroups, index=grouped.groups.keys()
        )
demo_counts_df = pd.DataFrame(demo_counts)

# =========== СБОРКА ПРОФИЛЯ ===========
profile_data = pd.concat(
    [
        profile_means,
        profile_counts,
        dominant_counts.rename("Ведущая установка"),
        signvalue_counts.rename("Метафора ИИ"),
        freq_counts.rename("Частота использования ИИ"),
        demo_counts_df.rename(
            columns={
                "Q2_AgeGroup": "Возраст",
                "Q3_Gender": "Пол",
                "Q5_MaritalStatus": "Семейное положение",
                "Q6_ChildrenStatus": "Наличие детей",
                "Q7_EducationLevel": "Образование",
                "Q9_Position": "Должность",
                "Q11_IncomeLevel": "Уровень дохода",
                "Q12_WellbeingLevel": "Благосостояние",
                "Q4_LocationType": "Тип населённого пункта",
            }
        ),
    ],
    axis=1,
)

# Если индекс — номер кластера (0, 1, 2...), иначе profile_data.reset_index(inplace=True)
profile_data["Название кластера"] = profile_data.index.map(
    lambda x: cluster_titles.get(x, {}).get("Название", "Без названия")
)
profile_data["Резюме"] = profile_data.index.map(
    lambda x: cluster_titles.get(x, {}).get("Резюме", "")
)


# =========== ОБРАТНОЕ ПРЕОБРАЗОВАНИЕ ДЕМОГРАФИИ ===========
mapping_dicts = {
    "Возраст": age_map,
    "Пол": gender_map,
    "Семейное положение": marital_map,
    "Наличие детей": children_map,
    "Образование": education_map,
    "Должность": position_map_simple,
    "Уровень дохода": income_map,
    "Благосостояние": wellbeing_map,
    "Тип населённого пункта": location_map,
    "Частота использования ИИ": ai_usage_freq_map,
}

for col, map_dict in mapping_dicts.items():
    if col in profile_data.columns:
        inv_map = {v: k for k, v in map_dict.items()}
        profile_data[col] = profile_data[col].apply(
            lambda x: (
                ", ".join(
                    inv_map.get(float(k), str(k)) + f": {v}"
                    for k, v in (item.split(": ") for item in x.split(", "))
                )
                if isinstance(x, str)
                else x
            )
        )
cols = ["Название кластера", "Резюме"] + [
    c for c in profile_data.columns if c not in ["Название кластера", "Резюме"]
]
profile_data = profile_data[cols]

print("--- Итоговый профиль по кластерам ---")
display(profile_data)

# --- Экспорт в Excel ---
profile_data.to_excel("profile_data_clusters_final.xlsx", index=True)

--- Итоговый профиль по кластерам ---


,Название кластера,Резюме,Практическое использование и влияние ИИ,"Доверие, антропоморфизм и социальное принятие ИИ",Знание и опыт взаимодействия с ИИ,Готовность делиться личной информацией и открытость,Сопротивление изменениям и тревожность по поводу ИИ,Готовность к обучению и освоению новых технологий,"Ожидания эффективности, контроля и безопасности",Мотивация и выгоды от использования ИИ,Тревожность и опасения по поводу рисков ИИ,N,Ведущая установка,Метафора ИИ,Частота использования ИИ,Возраст,Пол,Семейное положение,Наличие детей,Образование,Должность,Уровень дохода,Благосостояние,Тип населённого пункта
Cluster,,,,,,,,,,,,,,,,,,,,,,,,
0,Осторожные теоретики,"Среднее доверие (F2=0.10) при низком использовании (F1=-1.20). Воспринимают ИИ как 'Инструмент' (121). Основная группа: 45-64 года (104 чел.), с в...",-1.20,0.10,0.67,0.42,-0.06,-0.41,0.25,-0.26,0.76,221,"Стремление к развитию и интересу (Позитивные установки): 137, Негативные эмоции, избегают обсуждений с ИИ: 63, Фокус на контроле, прозрачности и б...","Инструмент: 91, Партнер: 66, Велосипед для разума: 58, Демон в процессоре: 4, Джинн из бутылки: 2","иногда (несколько раз в месяц): 97, часто (несколько раз в неделю): 77, редко (раз в несколько месяцев): 26, очень часто (ежедневно): 13, никогда: 6","45 - 64 лет: 104, 25 - 34 лет: 52, 35 - 44 лет: 52, 18 - 24 года: 13","женский: 126, мужской: 95","состою в браке / гражданском браке / отношениях: 139, холост/разведен/вдовствую: 82","нет: 81, один ребенок: 79, двое детей: 50, трое и более детей: 11","высшее: 142, среднее специальное: 64, среднее общее и ниже: 15","специалист: 101, руководитель отдела: 41, менеджер: 11, предприниматель / владелец бизнеса: 8, директор / исполнительный директор: 4, стажер: 4, п...","50 001 - 100 000 р: 94, до 50 000 р: 93, 100 001 - 150 000 р: 21, 200 001 - 400 000 р: 8, 150 001 - 200 000 р: 4, 400 001 р и выше: 1","на бытовую технику хватает, но покупка автомобиля вызывает затруднения: 116, на одежду хватает, но покупка бытовой техники вызывает затруднение: 7...","город с населением от 50 тыс до 1 млн чел: 86, город-миллионник, кроме москвы и санкт-петербурга: 74, москва, санкт-петербург: 61"
1,Критичные скептики,Крайне низкие ожидания безопасности (F7=-2.02). Активные мужчины (106 чел.) с низкой частотой использования. 52% имеют доход до 100 тыс.р.,-0.63,0.09,-0.62,-0.62,0.02,-0.24,-2.02,0.03,-0.46,184,"Стремление к развитию и интересу (Позитивные установки): 121, Негативные эмоции, избегают обсуждений с ИИ: 39, Фокус на контроле, прозрачности и б...","Инструмент: 100, Партнер: 45, Велосипед для разума: 32, Демон в процессоре: 5, Джинн из бутылки: 2","иногда (несколько раз в месяц): 66, редко (раз в несколько месяцев): 53, часто (несколько раз в неделю): 28, никогда: 14, очень часто (ежедневно): 8","45 - 64 лет: 69, 35 - 44 лет: 48, 25 - 34 лет: 48, 18 - 24 года: 19","мужской: 106, женский: 78","состою в браке / гражданском браке / отношениях: 115, холост/разведен/вдовствую: 69","нет: 76, один ребенок: 61, двое детей: 41, трое и более детей: 6","высшее: 96, среднее специальное: 65, среднее общее и ниже: 23","специалист: 97, менеджер: 18, руководитель отдела: 18, предприниматель / владелец бизнеса: 5, студент: 4, преподаватель / научный сотрудник: 3, ди...","50 001 - 100 000 р: 75, до 50 000 р: 69, 100 001 - 150 000 р: 29, 150 001 - 200 000 р: 7, 400 001 р и выше: 3, 200 001 - 400 000 р: 1","на бытовую технику хватает, но покупка автомобиля вызывает затруднения: 89, на одежду хватает, но покупка бытовой техники вызывает затруднение: 64...","город с населением от 50 тыс до 1 млн чел: 69, москва, санкт-петербург: 58, город-миллионник, кроме москвы и санкт-петербурга: 57"
2,Опытные скептики,"Активное использование (F1=0.67) без доверия (F2=-0.95). 113 мужчин vs 105 женщин. 55% состоят в браке, 40% имеют детей.",0.67,-0.95,-0.68,0.15,0.09,-0.15,0.33,-1.42,-0.03,218,"Стремление к развитию и интересу (Позитивные установки): 133, Испытывают трудности, сопротивляются новому: 76, Негативные эмоции, 

## Интерпретация кластеров: психологические портреты пользователей ИИ

### Кластер 0: Осторожные традиционалисты (n= 221)

Этот кластер — люди, которые **теоретически признают ценность ИИ** (F2 ≈ 0.1), но не видят для себя в нём практической выгоды (F8 ≈ -0.26) и используют его мало (F1 ≈ -1.20). Для них ИИ — это прежде всего "Инструмент", реже — "Велосипед для разума", то есть нечто, что может помочь, но не является необходимым или желанным спутником жизни. Они чаще испытывают внутренний конфликт: хотели бы развиваться, но боятся перемен, что подтверждается высокой долей установок "Стремление к развитию и интересу" и "Негативные эмоции, избегают обсуждений с ИИ".

**Соцдем:** Преимущественно 45-64 года, женский перевес, большинство состоят в браке, имеют высшее образование, средний доход.  
**Паттерн:** Классическая группа "барьера первого шага": они не против ИИ, но не готовы менять устоявшийся уклад ради его освоения.  
**Метафора:** ИИ — "инструмент", который лежит на полке, но редко используется.

---

### Кластер 1: Скептики без опоры (n= 184)

Главная черта — **крайне низкие ожидания безопасности и контроля** (F7 ≈ -2.02), что делает их недоверчивыми к ИИ. Они хотят развиваться, но не верят, что ИИ — надёжный помощник. Используют ИИ редко, воспринимают его исключительно как "Инструмент", без эмоциональной окраски. В кластере больше мужчин, что согласуется с данными о мужском техно-скептицизме: рациональная критика, а не страх.

**Соцдем:** Мужчины, 45-64 и 25-44 года, часто в браке, средний доход.  
**Паттерн:** Используют ИИ только по необходимости, не доверяют автоматизации, предпочитают ручной контроль.  
**Метафора:** "Инструмент" — холодный, не вызывающий доверия.

---

### Кластер 2: Прагматичные контролёры (n= 218)

Здесь собраны **активные пользователи ИИ** (F1 ≈ 0.67), которые видят в нём конкретную выгоду (F8 ≈ -1.42), но не склонны доверять или очеловечивать технологии (F2 ≈ -0.95). Они требуют от ИИ прозрачности и контролируемости, что видно по установке "Фокус на контроле, прозрачности и безопасности". Для них ИИ — это рабочий инструмент, который должен быть предсказуемым и хорошо управляемым.

**Соцдем:** 45-64 года, гендерный баланс, большинство в браке, часто с детьми.  
**Паттерн:** Используют ИИ для решения конкретных задач, не ждут от него "человечности", но хотят полного контроля.  
**Метафора:** "Инструмент" и "Велосипед для разума" — акцент на пользе и эффективности, а не на эмоциях.

---

### Кластер 3: Тревожные традиционалисты (n= 178)

Этот кластер — **люди, которые активно используют ИИ** (F1 ≈ 1.21), но **категорически против его антропоморфизации** (F2 ≈ -1.55). Они воспринимают ИИ как "Велосипед для разума", но среди них выше, чем в других кластерах, доля метафоры "Демон в процессоре", что говорит о внутренней тревоге и амбивалентности. Они используют ИИ скорее вынужденно, например, из-за требований работы или среды (много респондентов из мегаполисов).

**Соцдем:** Москва/СПб, мужчины и женщины, часто руководители.  
**Паттерн:** Используют ИИ, но не доверяют ему, боятся потери контроля и "очеловечивания" технологий.  
**Метафора:** "Велосипед для разума" с примесью "демонических" ассоциаций.

---

### Кластер 4: Продвинутые энтузиасты (n= 143)

Это **самые активные и позитивно настроенные пользователи ИИ** (F1 ≈ -0.83, F2 ≈ 0.04, F5 ≈ 0.55). Они воспринимают ИИ как "Партнёра" и "Велосипед для разума", не боятся изменений и быстро интегрируют новые технологии в жизнь. Для них ИИ — помощник, с которым можно экспериментировать. Часто отмечают финансовую устойчивость ("на автомобиль хватает"), что снижает страхи перед инновациями.

**Соцдем:** Молодые и зрелые, гендерный баланс, высокий уровень образования.  
**Паттерн:** Готовы пробовать новое, не боятся ошибок, видят в ИИ союзника.  
**Метафора:** "Партнёр" и "Велосипед для разума" — ИИ как расширение возможностей.

---

### Кластер 5: Тревожные экспериментаторы (n= 213)

Здесь — **люди с высокой тревожностью** (F5 ≈ 0.14), которые при этом активно используют ИИ (F1 ≈ -1.11), но слабо понимают его принципы (F3 ≈ 0.43). Это молодые мужчины, которые вынуждены осваивать ИИ ради конкурентоспособности, но внутренне сопротивляются и испытывают дискомфорт. Метафоры "Партнёр" и "Инструмент" встречаются почти одинаково часто — они не определились, кем для них является ИИ.

**Соцдем:** 25-44 года, мужчины, часто с детьми, средний доход.  
**Паттерн:** Используют ИИ "через силу", тревожатся из-за недостатка знаний.  
**Метафора:** "Инструмент", иногда "Партнёр" — двойственность отношения.

---

### Кластер 6: Сдержанные практики (n= 100)

**Самый большой кластер** — люди, которые мало используют ИИ (F1 ≈ 0.87), не склонны делиться личной информацией (F4 ≈ 0.01), но проявляют интерес к теме. Их установки — "Негативные эмоции" и "Стремление к развитию" — говорят о внутреннем конфликте между интересом и опасением. Для них ИИ — ограниченный инструмент, который не заслуживает особого доверия.

**Соцдем:** Женщины старшего возраста, среднее образование, средний доход.  
**Паттерн:** Интересуются ИИ, но не готовы активно использовать, предпочитают наблюдать.  
**Метафора:** "Инструмент" — что-то полезное, но не близкое.

---

### Кластер 7: Прагматичные, но осторожные (n= 177)

**Редко используют ИИ** (F1 ≈ -0.85), но готовы делиться информацией (F4 ≈ -1.85), не испытывают тревоги по поводу рисков (F9 ≈ 0.76). Используют ИИ функционально, не вовлекаясь эмоционально. Чаще женщины, много из Москвы и крупных городов.

**Соцдем:** Женщины, 45-64, семейные, средний доход.  
**Паттерн:** Используют ИИ по необходимости, не испытывают страха, но и не видят в нём ценности.  
**Метафора:** "Инструмент" — утилитарный, без эмоций.

---

### Кластер 8: Тревожные новаторы (n= 234)

Уникальный профиль: **открыты к обмену информацией** (F4 ≈ 1.07), стремятся к обучению (F6 ≈ 0.07), видят выгоды (F8 ≈ 0.86), но тревожатся о рисках (F9 ≈ -0.00). Молодые, часто из мегаполисов, балансируют между метафорами "Инструмент" и "Партнёр". Это "ранние последователи" с высокой тревожностью.

**Соцдем:** 25-34 года, мужчины и женщины, (!) руководители.  
**Паттерн:** Быстро осваивают новое, но опасаются последствий.  
**Метафора:** "Велосипед для разума" и "Инструмент" — ИИ как источник возможностей и тревог.

---

### Кластер 9: Встревоженные традиционалисты (n= 97)

**Мало используют ИИ** (F1 ≈ 0.81), но теоретически знают о нём (F3 ≈ -0.70), ожидают контроля и безопасности (F7 ≈ 0.48), испытывают высокую тревожность (F9 ≈ -0.06). Вынуждены использовать ИИ по профессиональной необходимости, часто руководители.

**Соцдем:** 45-64, мужчины и женщины, руководители, средний доход.  
**Паттерн:** Используют ИИ "через силу", опасаются рисков, но не могут отказаться от технологий.  
**Метафора:** "Инструмент" — необходимый, но не любимый.

---

### Кластер 10: Сбалансированные прагматики (n= 157)

**Умеренное использование** (F1 ≈ 0.86), фокус на контроле и безопасности, высокая доля метафоры "Велосипед для разума". Это люди, которые нашли баланс между пользой и рисками ИИ, используют его осознанно и без излишней тревоги.

**Соцдем:** 45-64, женщины, специалисты, средний доход, регионы.  
**Паттерн:** Используют ИИ там, где это удобно, не испытывают ни страха, ни эйфории.  
**Метафора:** "Велосипед для разума" — ИИ как расширение возможностей, но не как самостоятельный агент.

---

## Итоговые тенденции

- **Восприятие ИИ через метафору** (инструмент, партнёр, велосипед для разума) — ключ к пониманию установки к технологиям. https://arxiv.org/abs/2008.02311 и https://arxiv.org/html/2501.18045v1
- **Тревожность и сопротивление** чаще встречаются у тех, кто вынужден использовать ИИ по работе или под давлением среды.
- **Прагматизм** — доминирующая стратегия: большинство видит в ИИ инструмент, а не самостоятельного агента.
- **Гендерные и возрастные различия**: мужчины чаще скептики или тревожные экспериментаторы; женщины — осторожные прагматики и наблюдатели. Молодёжь быстрее осваивает ИИ, но чаще тревожится о последствиях.

**Практический вывод:**  
Разные группы требуют разных стратегий внедрения и коммуникации: одних нужно убеждать в безопасности, другим — показывать выгоды, третьим — давать простые и понятные сценарии применения. Только так можно преодолеть барьер недоверия и сделать ИИ частью повседневной жизни.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import qualitative
from scipy.spatial.distance import pdist, squareform

cluster_names = {
    0: "Осторожные теоретики",
    1: "Критичные скептики",
    2: "Опытные скептики",
    3: "Активные прагматики",
    4: "Тревожные минималисты",
    5: "Активные сомневающиеся",
    6: "Малоактивные наблюдатели",
    7: "Осторожные коммуникаторы",
    8: "Инновационные реалисты",
    9: "Вынужденные пользователи",
    10: "Сбалансированные прагматики",
}

# Если factor_names и factor_cols заранее заданы:
factor_labels = [factor_names.get(col, col) for col in factor_cols]

# 1. Средние по факторам
cluster_means = df_cluster.groupby("Cluster")[factor_cols].mean().round(2)
counts = df_cluster["Cluster"].value_counts().sort_index()
names = {k: f"Кластер {k}: {cluster_names.get(k, '')}" for k in cluster_means.index}

# 2. Находим полярные (максимально разные) кластеры
dist_matrix = squareform(pdist(cluster_means.values, metric="euclidean"))
pair_indices = np.unravel_index(
    np.argsort(dist_matrix.ravel())[::-1], dist_matrix.shape
)
polars = set()
for i, j in zip(pair_indices[0], pair_indices[1]):
    if i != j:
        polars.add(i)
        polars.add(j)
    if len(polars) >= 4:
        break
polars = list(polars)

palette = (qualitative.Dark24 + qualitative.Light24 + qualitative.Pastel)[0:17]

# 3. RADAR для максимального контраста
fig = go.Figure()
for idx in polars:
    row = cluster_means.iloc[idx]
    cluster = cluster_means.index[idx]
    color = palette[int(cluster) % len(palette)]
    fig.add_trace(
        go.Scatterpolar(
            r=row.values,
            theta=factor_labels,
            name=f"{names[cluster]} (N={counts[cluster]})",
            line=dict(width=3, color=color),
            opacity=0.9,
            hovertemplate="<b>%{theta}</b><br>Значение: %{r:.2f}<extra></extra>",
        )
    )
fig.update_layout(
    title="Максимально контрастные кластеры (по профилю факторов)",
    polar=dict(
        radialaxis=dict(
            range=[cluster_means.min().min() - 0.5, cluster_means.max().max() + 0.5],
            gridcolor="lightgray",
            linecolor="gray",
        ),
        angularaxis=dict(gridcolor="lightgray", linecolor="gray"),
    ),
    template="plotly_white",
    width=1100,
    height=800,
    legend=dict(font=dict(size=12)),
)
fig.show()

# 4. RADAR для всех кластеров по группам по n
n_per_group = 11
cluster_indices = list(cluster_means.index)
n_groups = int(np.ceil(len(cluster_indices) / n_per_group))

for g in range(n_groups):
    group = cluster_indices[g * n_per_group : (g + 1) * n_per_group]
    fig = go.Figure()
    for idx in group:
        row = cluster_means.loc[idx]
        color = palette[int(idx) % len(palette)]
        fig.add_trace(
            go.Scatterpolar(
                r=row.values,
                theta=factor_labels,
                name=f"{names[idx]} (N={counts[idx]})",
                line=dict(width=2, color=color),
                opacity=0.8,
            )
        )
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                range=[cluster_means.min().min() - 0.5, cluster_means.max().max() + 0.5]
            ),
            angularaxis=dict(gridcolor="lightgray", linecolor="gray"),
        ),
        template="plotly_white",
        width=1920,
        height=1080,
        legend=dict(font=dict(size=11)),
    )
    fig.show()

In [ ]:
import numpy as np
import pandas as pd
import xlsxwriter
from xlsxwriter.utility import xl_col_to_name

# --- Не делаем drop_duplicates, если не надо ---

# Дропаем только колонки, где все значения NaN
df_to_save = df_final_portraits.dropna(axis=1, how="all").copy()

excel_path = "df_full.xlsx"

with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:
    df_to_save.to_excel(writer, index=False, sheet_name="Sheet1")
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]

    num_cols = [
        col
        for col in df_to_save.select_dtypes(include="number").columns
        if col != "participant_id"
    ]

    for col in num_cols:
        data = df_to_save[col].dropna()
        # Фильтруем дихотомии (ровно два уникальных значения: 0 и 1, в любом порядке)
        uniques = set(data.unique())
        if len(uniques) == 2 and uniques <= {0, 1}:
            continue  # Пропускаем бинарные переменные

        col_idx = df_to_save.columns.get_loc(col)
        col_letter = xl_col_to_name(col_idx)

        if data.empty:
            continue

        # 10 и 90 процентиль
        perc10 = np.percentile(data, 10)
        perc90 = np.percentile(data, 90)

        col_range = f"{col_letter}2:{col_letter}{len(df_to_save)+1}"

        worksheet.conditional_format(
            col_range,
            {
                "type": "3_color_scale",
                "min_type": "num",
                "min_value": perc10,
                "min_color": "#F8696B",  # красный
                "mid_type": "percentile",
                "mid_value": 50,
                "mid_color": "#FFFFFF",  # белый
                "max_type": "num",
                "max_value": perc90,
                "max_color": "#63BE7B",  # зелёный
            },
        )

print(
    f"Файл с df_processed (без пустых колонок, без бинарной окраски) сохранён: {excel_path}"
)

ModuleNotFoundError: No module named 'xlsxwriter'